In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import string

import re

import json

In [2]:
import nltk
nltk.download('words')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package words to /home/cdsw/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:

import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:
def embed(input):
    return model(input)

In [5]:
words = set(nltk.corpus.words.words())

In [6]:
def df_lambda_detect_not_words(df, column_name, new_column):
    Lem = WordNetLemmatizer()
    df[new_column] = df[column_name].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if Lem.lemmatize(w.lower()) not in words))
    return df

In [7]:
def df_lambda_remove_punctuation(df, column_name, new_column):
    df[new_column] = df[column_name].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return df

In [8]:
def df_lambda_replace_characters(df, column_name, new_column, dictionary):
    df[new_column] = df[column_name].apply(lambda x: replace_words(x, dictionary))       
    return df


In [9]:
def replace_words(strtext, dictionary):
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in dictionary.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], strtext)
    return result

In [10]:
str_state = 'GA'
file = 'agent_comments_sales_'+str_state
column = 'clean_publicremarks'

noun_sentence_df = pd.read_feather('noun_sentences/' + file + '_withembeddings_erros_' + column + '_nounsentences_sentence_embedding.fea')



In [11]:
with open("replacement_dictionary_ordered.json") as json_file:
    replacement_dictionary_2 = json.load(json_file)



In [12]:
replacement_dictionary_2

{'lakedigit poolsdigit courtsgolf courseclub houseparksplaygrounds': 'lake pools courts golf course club housepark playgrounds',
 'digitdigitdigitdigit digitdigitdigitdigit': 'digitdigitdigitdigit digitdigitdigitdigit',
 'digitdigitdigitdigitdigitdigitdigitdigit': 'number',
 'digitpm digitnumberdigitdigitdigitdigit': 'number number',
 'digitnumberdigitdigitdigitdigit digitpm': 'number number pm',
 'digitdigitdigitk digitdigitdigitdigit': 'number number',
 'digitdigitdigitdigit digitdigitdigitk': 'number number',
 'digitdigitdigitdigitdigitdigitdigit': 'number - number - number',
 'digitdigitdigits digitdigitdigits': 'number number',
 'digitbeddigitdigitbath digitstory': 'digitbeddigitdigitbath digitstory',
 'digitdigitth digitdigitdigitdigit': 'digitdigitth digitdigitdigitdigit',
 'digitdigitdigitk digitdigitdigitk': 'number number',
 'digitdigitxdigitdigitxdigitdigit': 'number - number - number',
 'digitdigitk digitdigitdigitdigit': 'number number',
 'digitdigitdigitkdigitdigitdigitk'

In [13]:
noun_sentence_df_20 = noun_sentence_df[noun_sentence_df['count']>20]

In [14]:
noun_sentence_df_20 = df_lambda_remove_punctuation(noun_sentence_df_20, 'noun_sentence', 'noun_sentence_no_punct')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
noun_sentence_df_20 = df_lambda_replace_characters(noun_sentence_df_20,  'noun_sentence_no_punct', 
                                                   'noun_sentence_corrections', replacement_dictionary_2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
noun_sentence_df_20 = df_lambda_detect_not_words(noun_sentence_df_20, 'noun_sentence_corrections', 'not_words')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
noun_sentence_df_20

,noun_sentence,count,AVM_Error,AVM_Error_List,Listing_Error,Listing_Error_List,Listingid_List,AVM_Error_Average,Listing_Error_Average,AVM_Error_Indicators,Listing_Error_Indicators,noun_sentence_embeddings,noun_sentence_vaderpolarity,noun_sentence_no_punct,noun_sentence_corrections,not_words
63607,studs renovation,31,9.744215,"[0.4103665640631011, 0.41728319999999997, 0.49...",-0.735297,"[0.0, 0.00028571428571428574, 0.06693333333333...","[197894660, 250604412, 249157137, 222400238, 2...",0.314330,-0.023719,"[0.31432952538041775, 0.04068285195361636, -0....","[-0.02371925683439437, 0.0020194508549551816, ...","[0.052691612, 0.03693112, -0.049397767, -0.008...","[0.0, 1.0, 0.0, 0.0]",studs renovation,studs renovation,
83996,shower surrounds,28,7.905244,"[0.4387752777777778, 0.17622340350877197, 0.27...",-1.544048,"[-0.055, -0.08421052631578947, -0.019078947368...","[132872201, 6731906, 4232620, 5329671, 1258112...",0.282330,-0.055145,"[0.28233013942341034, 0.0424224129690018, -0.4...","[-0.05514457781159121, 0.0024243273137415197, ...","[-0.014650434, 0.07090296, 0.0504117, 0.011439...","[0.0, 1.0, 0.0, 0.0]",shower surrounds,shower surrounds,
92676,lee+white development,28,7.584127,"[0.44474872174879587, 0.5014364032697548, 0.09...",-2.173541,"[0.0, -0.00790190735694823, -0.029117647058823...","[211308616, 127087241, 133213133, 171564991, 1...",0.270862,-0.077626,"[0.2708616696841231, 0.06026944254415941, -0.7...","[-0.07762644975555874, 0.01239395576734469, -2...","[0.04612677, 0.00079587736, 0.043401964, -0.04...","[0.0, 1.0, 0.0, 0.0]",leewhite development,leewhite development,leewhite
113989,monday night brewery,21,5.171616,"[-0.02238924855491331, 0.5014364032697548, -0....",-0.625781,"[-0.06907514450867051, -0.00790190735694823, 0...","[200858826, 127087241, 172189751, 133213133, 1...",0.246267,-0.029799,"[0.2462674300702824, 0.09033614418445898, -1.5...","[-0.029799101674654667, 0.004036221684828119, ...","[0.060026523, -0.031776395, -0.07081793, -0.06...","[0.0, 1.0, 0.0, 0.0]",monday night brewery,monday night brewery,monday
116777,gorgeous bungalow,24,5.846151,"[0.06999810457516331, 0.4551995135135135, 0.29...",-0.840031,"[0.05555555555555555, -0.08054054054054054, -0...","[142728061, 210231641, 209961967, 152933557, 1...",0.243590,-0.035001,"[0.24358962556385214, 0.045847676984274735, -0...","[-0.035001307484190024, 0.006301288135261846, ...","[-0.020439927, -0.02788046, -0.017497879, 0.00...","[0.0, 0.2, 0.8, 0.6124]",gorgeous bungalow,gorgeous bungalow,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825189,utilities inspections,106,-52.553157,"[0.011213698630137103, -0.44224117647058825, 0...",-0.803344,"[0.043835616438356165, -0.20452488687782805, -...","[132883912, 4158664, 3640049, 140201011, 13941...",-0.495785,-0.007579,"[-0.4957845039683316, 0.2072645364563917, -0.0...","[-0.00757871387996991, 0.03244321830158484, -0...","[-0.011067669, -0.0022810258, 0.023981124, -0....","[0.0, 1.0, 0.0, 0.0]",utilities inspections,utilities inspections,
1825484,available inspections,25,-12.459462,"[-0.7051311666666666, -0.16786450000000008, -0...",0.214399,"[0.0016666666666666668, 0.125, 0.0, -0.149, 0....","[150636167, 173343008, 176532508, 187777456, 2...",-0.498378,0.008576,"[-0.4983784993571633, 0.08701769936855712, 0.2...","[0.008575964698432173, 0.019177693420631702, -...","[-0.029419353, 0.009323144, 0.0018775914, 0.05...","[0.0, 1.0, 0.0, 0.0]",available inspections,available inspections,
1826737,good potential,46,-23.538884,"[0.2837384191176471, -0.5405605, -0.3893973288...",-3.272146,"[-0.05606617647058824, -0.165, 0.0, -0.2153846...","[127085835, 171628048, 140951488, 180510904, 1...",-0.511715,-0.071134,"[-0.5117148606561, 0.1808759281132834, -0.3305...","[-0.0711336047468601, 0.026612259664944012, -0...","[0.006121108, -0.09186122, -0.049228374, -0.04...","[0.0, 0.256, 0.744, 0.4404]",good potential,good potential,
1830384,super corporate value,21,-11.603720,"[0.2306381481481481, -0.3187421

In [18]:
columns = ['noun_sentence_corrections']

In [19]:
import time
for column in columns:
    print(column)
    tic = time.time()
    new_column = column + '_embeddings'
    noun_sentence_df_20[new_column] = noun_sentence_df_20[column].apply(lambda x: embed([x]).numpy()[0]) 
    toc = time.time()
    print(toc - tic)

noun_sentence_corrections
156.04036927223206


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
noun_sentence_df_20

,noun_sentence,count,AVM_Error,AVM_Error_List,Listing_Error,Listing_Error_List,Listingid_List,AVM_Error_Average,Listing_Error_Average,AVM_Error_Indicators,Listing_Error_Indicators,noun_sentence_embeddings,noun_sentence_vaderpolarity,noun_sentence_no_punct,noun_sentence_corrections,not_words,noun_sentence_corrections_embeddings
63607,studs renovation,31,9.744215,"[0.4103665640631011, 0.41728319999999997, 0.49...",-0.735297,"[0.0, 0.00028571428571428574, 0.06693333333333...","[197894660, 250604412, 249157137, 222400238, 2...",0.314330,-0.023719,"[0.31432952538041775, 0.04068285195361636, -0....","[-0.02371925683439437, 0.0020194508549551816, ...","[0.052691612, 0.03693112, -0.049397767, -0.008...","[0.0, 1.0, 0.0, 0.0]",studs renovation,studs renovation,,"[0.052691612, 0.03693112, -0.049397767, -0.008..."
83996,shower surrounds,28,7.905244,"[0.4387752777777778, 0.17622340350877197, 0.27...",-1.544048,"[-0.055, -0.08421052631578947, -0.019078947368...","[132872201, 6731906, 4232620, 5329671, 1258112...",0.282330,-0.055145,"[0.28233013942341034, 0.0424224129690018, -0.4...","[-0.05514457781159121, 0.0024243273137415197, ...","[-0.014650434, 0.07090296, 0.0504117, 0.011439...","[0.0, 1.0, 0.0, 0.0]",shower surrounds,shower surrounds,,"[-0.014650434, 0.07090296, 0.0504117, 0.011439..."
92676,lee+white development,28,7.584127,"[0.44474872174879587, 0.5014364032697548, 0.09...",-2.173541,"[0.0, -0.00790190735694823, -0.029117647058823...","[211308616, 127087241, 133213133, 171564991, 1...",0.270862,-0.077626,"[0.2708616696841231, 0.06026944254415941, -0.7...","[-0.07762644975555874, 0.01239395576734469, -2...","[0.04612677, 0.00079587736, 0.043401964, -0.04...","[0.0, 1.0, 0.0, 0.0]",leewhite development,leewhite development,leewhite,"[0.03309278, 0.021012176, 0.017111761, -0.0294..."
113989,monday night brewery,21,5.171616,"[-0.02238924855491331, 0.5014364032697548, -0....",-0.625781,"[-0.06907514450867051, -0.00790190735694823, 0...","[200858826, 127087241, 172189751, 133213133, 1...",0.246267,-0.029799,"[0.2462674300702824, 0.09033614418445898, -1.5...","[-0.029799101674654667, 0.004036221684828119, ...","[0.060026523, -0.031776395, -0.07081793, -0.06...","[0.0, 1.0, 0.0, 0.0]",monday night brewery,monday night brewery,monday,"[0.060026523, -0.031776395, -0.07081793, -0.06..."
116777,gorgeous bungalow,24,5.846151,"[0.06999810457516331, 0.4551995135135135, 0.29...",-0.840031,"[0.05555555555555555, -0.08054054054054054, -0...","[142728061, 210231641, 209961967, 152933557, 1...",0.243590,-0.035001,"[0.24358962556385214, 0.045847676984274735, -0...","[-0.035001307484190024, 0.006301288135261846, ...","[-0.020439927, -0.02788046, -0.017497879, 0.00...","[0.0, 0.2, 0.8, 0.6124]",gorgeous bungalow,gorgeous bungalow,,"[-0.020439927, -0.02788046, -0.017497879, 0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825189,utilities inspections,106,-52.553157,"[0.011213698630137103, -0.44224117647058825, 0...",-0.803344,"[0.043835616438356165, -0.20452488687782805, -...","[132883912, 4158664, 3640049, 140201011, 13941...",-0.495785,-0.007579,"[-0.4957845039683316, 0.2072645364563917, -0.0...","[-0.00757871387996991, 0.03244321830158484, -0...","[-0.011067669, -0.0022810258, 0.023981124, -0....","[0.0, 1.0, 0.0, 0.0]",utilities inspections,utilities inspections,,"[-0.011067669, -0.0022810258, 0.023981124, -0...."
1825484,available inspections,25,-12.459462,"[-0.7051311666666666, -0.16786450000000008, -0...",0.214399,"[0.0016666666666666668, 0.125, 0.0, -0.149, 0....","[150636167, 173343008, 176532508, 187777456, 2...",-0.498378,0.008576,"[-0.4983784993571633, 0.08701769936855712, 0.2...","[0.008575964698432173, 0.019177693420631702, -...","[-0.029419353, 0.009323144, 0.0018775914, 0.05...","[0.0, 1.0, 0.0, 0.0]",available inspections,available inspections,,"[-0.029419353, 0.009323144, 0.0018775914, 0.05..."
1826737,good potential,46,-23.538884,"[0.2837384191176471, -0.5405605, -0.3893973288...",-3.272146,"[-0.0560661

In [21]:
noun_sentence_df_1000 = noun_sentence_df_20[noun_sentence_df_20['count']>1000]

In [22]:
noun_sentence_df_1000_corr = noun_sentence_df_1000[['noun_sentence_corrections', 'noun_sentence_corrections_embeddings']]
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.set_index('noun_sentence_corrections')

In [23]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index()

In [24]:
column = 'noun_sentence_corrections_embeddings'
nelements = (len(noun_sentence_df_1000_corr[column][0]))
#print(nelements)
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
#print(column_names)
split_df = pd.DataFrame(noun_sentence_df_1000_corr[column].tolist(), columns=column_names)


In [25]:
noun_sentence_df_1000_corr


,noun_sentence_corrections,noun_sentence_corrections_embeddings
0,new bathrooms,"[-0.011923872, -0.040925846, -0.060033374, -0...."
1,new vanities,"[-0.05858002, -0.078841336, 0.007744186, -0.04..."
2,quartz countertops,"[-0.030976344, 0.010525043, 0.008025839, -0.02..."
3,top bottom,"[-0.055551246, -0.050620284, 0.022666093, 0.04..."
4,new kitchen cabinets,"[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...
634,quick sale,"[-0.07425289, -0.07586487, 0.018565405, 0.0660..."
635,short sale,"[-0.07748554, -0.07403186, 0.06659468, -0.0200..."
636,great investment property,"[0.055392615, -0.065018825, 0.0022482187, -0.0..."
637,great investment opportunity,"[0.0669903, -0.08734736, -0.073891625, -0.0216..."


In [26]:
split_df

,noun_sentence_corrections_embeddings_0,noun_sentence_corrections_embeddings_1,noun_sentence_corrections_embeddings_2,noun_sentence_corrections_embeddings_3,noun_sentence_corrections_embeddings_4,noun_sentence_corrections_embeddings_5,noun_sentence_corrections_embeddings_6,noun_sentence_corrections_embeddings_7,noun_sentence_corrections_embeddings_8,noun_sentence_corrections_embeddings_9,...,noun_sentence_corrections_embeddings_502,noun_sentence_corrections_embeddings_503,noun_sentence_corrections_embeddings_504,noun_sentence_corrections_embeddings_505,noun_sentence_corrections_embeddings_506,noun_sentence_corrections_embeddings_507,noun_sentence_corrections_embeddings_508,noun_sentence_corrections_embeddings_509,noun_sentence_corrections_embeddings_510,noun_sentence_corrections_embeddings_511
0,-0.011924,-0.040926,-0.060033,-0.022761,-0.073656,0.081821,-0.005458,0.006407,-0.059856,-0.043086,...,-0.001005,-0.076426,0.062387,0.015463,0.041506,0.025877,-0.023413,-0.012487,0.038683,0.034699
1,-0.058580,-0.078841,0.007744,-0.043522,-0.080803,0.049983,0.030043,0.046446,-0.031561,-0.043473,...,0.014715,-0.103510,0.068328,0.001586,0.077301,0.024143,-0.044901,-0.042204,0.076957,-0.000690
2,-0.030976,0.010525,0.008026,-0.024324,0.003008,-0.040600,-0.004780,0.043914,-0.019643,0.012021,...,0.054266,-0.063503,0.074849,0.038589,-0.002148,0.058436,0.056435,-0.035219,0.034700,-0.030047
3,-0.055551,-0.050620,0.022666,0.040247,0.026183,-0.005721,0.053899,0.063238,-0.036279,0.046826,...,0.010186,-0.089421,-0.010154,-0.006709,0.056422,-0.038414,-0.005224,-0.043338,-0.012072,-0.013668
4,-0.002790,0.035919,-0.032957,-0.048845,-0.047378,0.038537,0.014815,0.012746,-0.070697,-0.063335,...,0.043098,-0.059954,0.015364,0.067849,0.037710,0.036774,-0.049861,0.025568,0.076711,0.014323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,-0.074253,-0.075865,0.018565,0.066049,0.017042,0.050962,0.057620,-0.029769,0.011815,-0.070450,...,0.049589,-0.093314,0.058984,0.062574,-0.003633,-0.041592,-0.084680,-0.012300,-0.002827,0.009741
635,-0.077486,-0.074032,0.066595,-0.020082,0.041439,0.016017,0.021577,-0.019751,-0.047435,-0.080104,...,-0.022587,-0.077284,0.045462,0.065289,0.032451,0.027428,-0.043622,-0.009963,0.015390,-0.036873
636,0.055393,-0.065019,0.002248,-0.037914,0.055723,0.012557,0.047077,-0.013009,-0.016286,-0.061209,...,-0.005474,-0.048618,0.027036,0.088642,-0.062457,0.081568,0.043043,-0.009728,-0.011720,0.019987
637,0.066990,-0.087347,-0.073892,-0.021683,0.045020,-0.000418,0.003835,-0.009434,0.002560,-0.048241,...,-0.033711,-0.078678,0.006397,0.080112,-0.026150,0.078106,0.007136,-0.022563,-0.029145,-0.029067


In [27]:
noun_sentence_df_1000_corr = pd.concat([noun_sentence_df_1000_corr, split_df], axis=1)
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns = [column])

In [28]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.set_index('noun_sentence_corrections')

In [29]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.T

In [30]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index()

In [31]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns=['index'])

In [32]:
noun_sentence_df_1000_corr

noun_sentence_corrections,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new cabinets,new windows,new driveway,subway tile backsplash,...,great value,short sale foreclosure,large home,private remarks,great opportunity,quick sale,short sale,great investment property,great investment opportunity,needs tender loving care
0,-0.011924,-0.058580,-0.030976,-0.055551,-0.002790,-0.062026,-0.013443,-0.009699,-0.004411,0.012699,...,-0.019999,-0.068759,-0.057489,-0.050721,0.032697,-0.074253,-0.077486,0.055393,0.066990,0.003912
1,-0.040926,-0.078841,0.010525,-0.050620,0.035919,-0.043769,-0.016529,-0.045838,-0.018443,-0.003868,...,-0.094221,-0.071003,-0.017712,0.071219,-0.087883,-0.075865,-0.074032,-0.065019,-0.087347,-0.042235
2,-0.060033,0.007744,0.008026,0.022666,-0.032957,-0.008107,-0.031754,-0.020702,-0.017115,-0.023148,...,-0.064668,0.053975,-0.036073,0.017914,-0.078332,0.018565,0.066595,0.002248,-0.073892,-0.019112
3,-0.022761,-0.043522,-0.024324,0.040247,-0.048845,-0.042315,-0.038782,-0.007243,-0.010116,-0.003280,...,0.004551,-0.039184,-0.001802,-0.030973,-0.022118,0.066049,-0.020082,-0.037914,-0.021683,0.034365
4,-0.073656,-0.080803,0.003008,0.026183,-0.047378,-0.052893,-0.071612,-0.046792,-0.033687,0.023922,...,0.069442,0.055099,0.065950,-0.041265,0.059580,0.017042,0.041439,0.055723,0.045020,0.024409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.025877,0.024143,0.058436,-0.038414,0.036774,0.033655,0.043721,0.056801,0.035004,0.074835,...,-0.026022,0.067697,-0.039932,0.023846,0.006058,-0.041592,0.027428,0.081568,0.078106,0.013642
508,-0.023413,-0.044901,0.056435,-0.005224,-0.049861,-0.027888,-0.038394,-0.033330,-0.015601,-0.050905,...,-0.025732,-0.037336,-0.016188,0.026418,-0.039742,-0.084680,-0.043622,0.043043,0.007136,0.064991
509,-0.012487,-0.042204,-0.035219,-0.043338,0.025568,-0.017498,0.041377,0.000409,-0.005241,0.020724,...,-0.047997,-0.042702,-0.037079,0.033496,0.000353,-0.012300,-0.009963,-0.009728,-0.022563,-0.017943
510,0.038683,0.076957,0.034700,-0.012072,0.076711,0.066418,0.076612,0.039703,0.007052,0.031845,...,-0.031651,0.026585,0.030498,-0.017686,0.000578,-0.002827,0.015390,-0.011720,-0.029145,-0.043296


In [33]:
correlations_df = noun_sentence_df_1000_corr.corr(method='pearson')

In [34]:
correlations_df

noun_sentence_corrections,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new cabinets,new windows,new driveway,subway tile backsplash,...,great value,short sale foreclosure,large home,private remarks,great opportunity,quick sale,short sale,great investment property,great investment opportunity,needs tender loving care
noun_sentence_corrections,,,,,,,,,,,,,,,,,,,,,
new bathrooms,1.000000,0.590982,0.135705,0.102463,0.451565,0.599612,0.549633,0.478538,0.446852,0.282209,...,0.134572,0.121494,0.243616,0.173221,0.205447,0.284246,0.190221,0.040568,0.073820,0.025418
new vanities,0.590982,1.000000,0.136238,0.200501,0.417032,0.578911,0.631558,0.505135,0.488338,0.124328,...,0.244444,0.150134,0.256693,0.142263,0.251338,0.269713,0.227607,0.074415,0.140209,0.091845
quartz countertops,0.135705,0.136238,1.000000,0.106559,0.381654,0.249013,0.304748,0.150571,0.219898,0.348028,...,0.008422,0.122790,0.143048,-0.042067,0.027060,0.097559,0.063355,0.111687,0.081435,0.178230
top bottom,0.102463,0.200501,0.106559,1.000000,0.099901,0.167759,0.141665,0.109949,0.138972,0.131350,...,0.210635,0.105647,0.358653,0.148216,0.081773,0.230263,0.209441,0.104573,0.095694,0.120745
new kitchen cabinets,0.451565,0.417032,0.381654,0.099901,1.000000,0.756371,0.816629,0.392157,0.450471,0.423458,...,0.116761,0.211597,0.194274,0.078456,0.068458,0.224576,0.191850,0.148849,0.041638,0.064804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
quick sale,0.284246,0.269713,0.097559,0.230263,0.224576,0.265469,0.315233,0.318442,0.353605,0.099706,...,0.382405,0.425293,0.333358,0.215735,0.391686,1.000000,0.612635,0.224054,0.256960,0.108490
short sale,0.190221,0.227607,0.063355,0.209441,0.191850,0.204232,0.246531,0.233890,0.274812,0.076319,...,0.258297,0.843862,0.312843,0.248084,0.220414,0.612635,1.000000,0.271750,0.190678,0.094276
great investment property,0.040568,0.074415,0.111687,0.104573,0.148849,0.163727,0.122736,0.153263,0.203115,0.080845,...,0.477144,0.278398,0.331347,0.042090,0.412551,0.224054,0.271750,1.000000,0.701701,0.162563


In [35]:
skill_group = {}
test_df = correlations_df.copy('deep')

while len(test_df) > 0:
    idd = test_df.index[0]
    print(idd)
    group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
    print(len(group))
    #print(noun_sentence_df_20_reindex.loc[group,'AVM_Error_Average'])
    skill_group[group[0]] = group
    test_df = test_df.drop(labels = group, axis = 1)
    test_df = test_df.drop(labels = group, axis = 0)

new bathrooms
1
new vanities
1
quartz countertops
1
top bottom
1
new kitchen cabinets
3
new kitchen
2
new windows
1
new driveway
1
subway tile backsplash
1
new toilets
1
new gutters
1
new deck
1
new stainless steel appliances
2
efficient windows
1
new floors
1
new hvac
2
private oasis
1
new light fixtures
1
custom cabinets
2
new fixtures
1
new stainless appliances
7
new granite counter tops
8
new exterior
1
number mumber
1
custom home
1
wine cooler
1
fire pit
1
new hardwood
3
new appliances
1
atlanta village
1
new tile
1
light fixtures
1
original hardwood floors
5
kitchen baths
1
new interior exterior paint
3
tile bathrooms
1
open concept
1
kitchen bathrooms
1
new carpet bedrooms
1
interior exterior
1
entire home
2
wine cellar
1
number
13
new construction
1
hvac system
2
gorgeous hardwood floors
2
water heater
3
tile shower
1
white cabinets
2
hot tub
1
gorgeous hardwoods
2
new roof
1
mud room
1
upgrades galore
1
digitdigitft
1
garden area
1
privacy fence
1
new countertops
1
digit
2
hug

In [36]:
len(skill_group)

456

In [37]:
skill_group

{'new bathrooms': ['new bathrooms'],
 'new vanities': ['new vanities'],
 'quartz countertops': ['quartz countertops'],
 'top bottom': ['top bottom'],
 'new kitchen cabinets': ['new kitchen cabinets',
  'new cabinets',
  'kitchen cabinets'],
 'new kitchen': ['new kitchen', 'new kitchen appliances'],
 'new windows': ['new windows'],
 'new driveway': ['new driveway'],
 'subway tile backsplash': ['subway tile backsplash'],
 'new toilets': ['new toilets'],
 'new gutters': ['new gutters'],
 'new deck': ['new deck'],
 'new stainless steel appliances': ['new stainless steel appliances',
  'new stainless steel appliances'],
 'efficient windows': ['efficient windows'],
 'new floors': ['new floors'],
 'new hvac': ['new hvac', 'new hvac system'],
 'private oasis': ['private oasis'],
 'new light fixtures': ['new light fixtures'],
 'custom cabinets': ['custom cabinets', 'custom cabinetry'],
 'new fixtures': ['new fixtures'],
 'new stainless appliances': ['new stainless appliances',
  'stainless stee

In [38]:
noun_sentence_df_50_1000 = noun_sentence_df_20[(noun_sentence_df_20['count']>50)
                                               &(noun_sentence_df_20['count']<=1000)]

In [39]:
len(noun_sentence_df_50_1000)

10639

In [40]:
noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000.set_index('noun_sentence_corrections')


In [41]:
noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000_reindex.loc[:,['noun_sentence_corrections_embeddings']]

In [42]:
noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000_reindex.reset_index()

In [43]:
noun_sentence_df_50_1000_reindex

,noun_sentence_corrections,noun_sentence_corrections_embeddings
0,luxurious baths,"[0.011624178, -0.01388437, 0.068723805, 0.0026..."
1,new shaker cabinets,"[0.032202173, 0.042373072, -0.0025964438, -0.0..."
2,chefs kitchen w stainless steel appliances,"[-0.0021265124, -0.015930299, 0.04821109, -0.0..."
3,tack room,"[-0.04527767, 0.074812554, 0.0010289665, 0.033..."
4,new drywall,"[-0.0008549273, 0.04306897, -0.009549875, -0.0..."
...,...,...
10634,seller requests buyer use seller,"[-0.071547754, -0.007990354, 0.044830173, 0.01..."
10635,great home investors,"[-0.021865588, -0.05759102, -0.037234746, -0.0..."
10636,needs repairs,"[-0.04354262, -0.037721384, 0.010220117, 0.044..."
10637,seller chooses attorney,"[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [44]:


column = 'noun_sentence_corrections_embeddings'
nelements = (len(noun_sentence_df_50_1000_reindex[column][0]))
#print(nelements)
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
#print(column_names)
split_df = pd.DataFrame(noun_sentence_df_50_1000_reindex[column].tolist(), columns=column_names)


In [45]:
split_df

,noun_sentence_corrections_embeddings_0,noun_sentence_corrections_embeddings_1,noun_sentence_corrections_embeddings_2,noun_sentence_corrections_embeddings_3,noun_sentence_corrections_embeddings_4,noun_sentence_corrections_embeddings_5,noun_sentence_corrections_embeddings_6,noun_sentence_corrections_embeddings_7,noun_sentence_corrections_embeddings_8,noun_sentence_corrections_embeddings_9,...,noun_sentence_corrections_embeddings_502,noun_sentence_corrections_embeddings_503,noun_sentence_corrections_embeddings_504,noun_sentence_corrections_embeddings_505,noun_sentence_corrections_embeddings_506,noun_sentence_corrections_embeddings_507,noun_sentence_corrections_embeddings_508,noun_sentence_corrections_embeddings_509,noun_sentence_corrections_embeddings_510,noun_sentence_corrections_embeddings_511
0,0.011624,-0.013884,0.068724,0.002632,0.019046,0.067914,-0.052874,-0.052268,-0.050814,0.047787,...,0.029364,-0.071449,0.038064,-0.039660,-0.029154,0.010361,-0.039594,-0.025802,0.035652,-0.027203
1,0.032202,0.042373,-0.002596,-0.037502,-0.024633,0.049036,0.020337,0.045200,-0.063089,-0.045580,...,0.044738,-0.070369,0.039176,0.023146,0.036038,0.040626,-0.063173,0.000080,0.047462,-0.008436
2,-0.002127,-0.015930,0.048211,-0.029201,0.074552,0.068436,0.022993,-0.030893,-0.031945,-0.034055,...,0.007764,0.052105,0.020797,0.050045,0.031504,0.007940,-0.037985,-0.023704,0.079927,-0.021342
3,-0.045278,0.074813,0.001029,0.033538,0.051842,-0.005769,0.011611,0.009372,0.051918,0.001941,...,0.051660,-0.092755,0.048564,0.039528,0.032542,-0.078584,0.003814,0.000022,0.081814,0.005120
4,-0.000855,0.043069,-0.009550,-0.034985,-0.025922,0.012465,0.060297,0.017276,0.014118,0.015962,...,0.021289,-0.073174,0.027352,0.002612,0.041522,0.045266,-0.027960,-0.020402,0.051779,0.062820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10634,-0.071548,-0.007990,0.044830,0.017913,0.083349,-0.045600,0.056955,-0.060445,-0.069979,-0.086626,...,0.043277,-0.044363,0.059579,-0.036826,0.080305,0.012657,0.046255,0.039482,0.028458,-0.035270
10635,-0.021866,-0.057591,-0.037235,-0.001225,0.006665,0.037057,0.030569,-0.053313,-0.065151,-0.066282,...,0.055516,-0.034472,0.080640,0.085126,-0.077353,0.086770,-0.001477,-0.013598,-0.020192,-0.014179
10636,-0.043543,-0.037721,0.010220,0.044749,0.024704,0.038501,-0.027768,0.012042,0.037737,0.028988,...,0.058412,-0.067837,0.051379,0.027381,-0.019979,-0.001432,0.059121,-0.054112,0.025230,-0.001131
10637,-0.058099,-0.012980,-0.021597,0.037381,0.016936,-0.007160,0.053621,-0.014170,0.031248,-0.095731,...,0.032972,-0.083984,-0.017622,0.011074,0.074720,0.077808,0.043726,0.029706,0.076158,-0.031519


In [46]:
noun_sentence_df_50_1000_reindex

,noun_sentence_corrections,noun_sentence_corrections_embeddings
0,luxurious baths,"[0.011624178, -0.01388437, 0.068723805, 0.0026..."
1,new shaker cabinets,"[0.032202173, 0.042373072, -0.0025964438, -0.0..."
2,chefs kitchen w stainless steel appliances,"[-0.0021265124, -0.015930299, 0.04821109, -0.0..."
3,tack room,"[-0.04527767, 0.074812554, 0.0010289665, 0.033..."
4,new drywall,"[-0.0008549273, 0.04306897, -0.009549875, -0.0..."
...,...,...
10634,seller requests buyer use seller,"[-0.071547754, -0.007990354, 0.044830173, 0.01..."
10635,great home investors,"[-0.021865588, -0.05759102, -0.037234746, -0.0..."
10636,needs repairs,"[-0.04354262, -0.037721384, 0.010220117, 0.044..."
10637,seller chooses attorney,"[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [47]:
split_df 

,noun_sentence_corrections_embeddings_0,noun_sentence_corrections_embeddings_1,noun_sentence_corrections_embeddings_2,noun_sentence_corrections_embeddings_3,noun_sentence_corrections_embeddings_4,noun_sentence_corrections_embeddings_5,noun_sentence_corrections_embeddings_6,noun_sentence_corrections_embeddings_7,noun_sentence_corrections_embeddings_8,noun_sentence_corrections_embeddings_9,...,noun_sentence_corrections_embeddings_502,noun_sentence_corrections_embeddings_503,noun_sentence_corrections_embeddings_504,noun_sentence_corrections_embeddings_505,noun_sentence_corrections_embeddings_506,noun_sentence_corrections_embeddings_507,noun_sentence_corrections_embeddings_508,noun_sentence_corrections_embeddings_509,noun_sentence_corrections_embeddings_510,noun_sentence_corrections_embeddings_511
0,0.011624,-0.013884,0.068724,0.002632,0.019046,0.067914,-0.052874,-0.052268,-0.050814,0.047787,...,0.029364,-0.071449,0.038064,-0.039660,-0.029154,0.010361,-0.039594,-0.025802,0.035652,-0.027203
1,0.032202,0.042373,-0.002596,-0.037502,-0.024633,0.049036,0.020337,0.045200,-0.063089,-0.045580,...,0.044738,-0.070369,0.039176,0.023146,0.036038,0.040626,-0.063173,0.000080,0.047462,-0.008436
2,-0.002127,-0.015930,0.048211,-0.029201,0.074552,0.068436,0.022993,-0.030893,-0.031945,-0.034055,...,0.007764,0.052105,0.020797,0.050045,0.031504,0.007940,-0.037985,-0.023704,0.079927,-0.021342
3,-0.045278,0.074813,0.001029,0.033538,0.051842,-0.005769,0.011611,0.009372,0.051918,0.001941,...,0.051660,-0.092755,0.048564,0.039528,0.032542,-0.078584,0.003814,0.000022,0.081814,0.005120
4,-0.000855,0.043069,-0.009550,-0.034985,-0.025922,0.012465,0.060297,0.017276,0.014118,0.015962,...,0.021289,-0.073174,0.027352,0.002612,0.041522,0.045266,-0.027960,-0.020402,0.051779,0.062820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10634,-0.071548,-0.007990,0.044830,0.017913,0.083349,-0.045600,0.056955,-0.060445,-0.069979,-0.086626,...,0.043277,-0.044363,0.059579,-0.036826,0.080305,0.012657,0.046255,0.039482,0.028458,-0.035270
10635,-0.021866,-0.057591,-0.037235,-0.001225,0.006665,0.037057,0.030569,-0.053313,-0.065151,-0.066282,...,0.055516,-0.034472,0.080640,0.085126,-0.077353,0.086770,-0.001477,-0.013598,-0.020192,-0.014179
10636,-0.043543,-0.037721,0.010220,0.044749,0.024704,0.038501,-0.027768,0.012042,0.037737,0.028988,...,0.058412,-0.067837,0.051379,0.027381,-0.019979,-0.001432,0.059121,-0.054112,0.025230,-0.001131
10637,-0.058099,-0.012980,-0.021597,0.037381,0.016936,-0.007160,0.053621,-0.014170,0.031248,-0.095731,...,0.032972,-0.083984,-0.017622,0.011074,0.074720,0.077808,0.043726,0.029706,0.076158,-0.031519


In [48]:
noun_sentence_df_50_1000_corr = pd.concat([noun_sentence_df_50_1000_reindex, split_df], axis=1)
noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.drop(columns = [column])

In [49]:
noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.set_index('noun_sentence_corrections')

In [50]:
noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.T

In [53]:
noun_sentence_df_50_1000_corr.head()

noun_sentence_corrections,luxurious baths,new shaker cabinets,chefs kitchen w stainless steel appliances,tack room,new drywall,white shaker cabinets,perkerson park,new hall bath,stall barn,new sheetrock,...,clean property,great fixer,cash sale,digitdigitdigitdigitdigitdigitdigitdigitdigit,excellent opportunity smart bargain hunter,seller requests buyer use seller,great home investors,needs repairs,seller chooses attorney,utilities inspections
noun_sentence_corrections_embeddings_0,0.011624,0.032202,-0.002127,-0.045278,-0.000855,0.027027,0.007650,-0.047220,0.034345,-0.035343,...,-0.041365,0.010111,-0.067054,0.060710,0.043231,-0.071548,-0.021866,-0.043543,-0.058099,-0.011068
noun_sentence_corrections_embeddings_1,-0.013884,0.042373,-0.015930,0.074813,0.043069,0.071275,0.051528,0.022811,0.044923,-0.007725,...,-0.043555,-0.015113,-0.054921,-0.026173,-0.085849,-0.007990,-0.057591,-0.037721,-0.012980,-0.002281
noun_sentence_corrections_embeddings_2,0.068724,-0.002596,0.048211,0.001029,-0.009550,0.009335,0.049954,-0.021367,-0.039344,0.042356,...,0.041377,-0.054446,0.041126,0.025103,-0.075059,0.044830,-0.037235,0.010220,-0.021597,0.023981
noun_sentence_corrections_embeddings_3,0.002632,-0.037502,-0.029201,0.033538,-0.034985,0.018112,0.022226,-0.041952,-0.008421,-0.035431,...,-0.023800,0.019810,0.010891,0.079969,-0.011753,0.017913,-0.001225,0.044749,0.037381,-0.018314
noun_sentence_corrections_embeddings_4,0.019046,-0.024633,0.074552,0.051842,-0.025922,0.060065,0.000760,-0.031940,0.066635,-0.044019,...,0.056672,0.070721,0.039114,0.014042,-0.036611,0.083349,0.006665,0.024704,0.016936,-0.066819


In [54]:
noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.reset_index()#.drop(columns=['index','level_0'])

In [55]:
noun_sentence_df_50_1000_corr

noun_sentence_corrections,index,luxurious baths,new shaker cabinets,chefs kitchen w stainless steel appliances,tack room,new drywall,white shaker cabinets,perkerson park,new hall bath,stall barn,...,clean property,great fixer,cash sale,digitdigitdigitdigitdigitdigitdigitdigitdigit,excellent opportunity smart bargain hunter,seller requests buyer use seller,great home investors,needs repairs,seller chooses attorney,utilities inspections
0,noun_sentence_corrections_embeddings_0,0.011624,0.032202,-0.002127,-0.045278,-0.000855,0.027027,0.007650,-0.047220,0.034345,...,-0.041365,0.010111,-0.067054,0.060710,0.043231,-0.071548,-0.021866,-0.043543,-0.058099,-0.011068
1,noun_sentence_corrections_embeddings_1,-0.013884,0.042373,-0.015930,0.074813,0.043069,0.071275,0.051528,0.022811,0.044923,...,-0.043555,-0.015113,-0.054921,-0.026173,-0.085849,-0.007990,-0.057591,-0.037721,-0.012980,-0.002281
2,noun_sentence_corrections_embeddings_2,0.068724,-0.002596,0.048211,0.001029,-0.009550,0.009335,0.049954,-0.021367,-0.039344,...,0.041377,-0.054446,0.041126,0.025103,-0.075059,0.044830,-0.037235,0.010220,-0.021597,0.023981
3,noun_sentence_corrections_embeddings_3,0.002632,-0.037502,-0.029201,0.033538,-0.034985,0.018112,0.022226,-0.041952,-0.008421,...,-0.023800,0.019810,0.010891,0.079969,-0.011753,0.017913,-0.001225,0.044749,0.037381,-0.018314
4,noun_sentence_corrections_embeddings_4,0.019046,-0.024633,0.074552,0.051842,-0.025922,0.060065,0.000760,-0.031940,0.066635,...,0.056672,0.070721,0.039114,0.014042,-0.036611,0.083349,0.006665,0.024704,0.016936,-0.066819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,noun_sentence_corrections_embeddings_507,0.010361,0.040626,0.007940,-0.078584,0.045266,0.025759,-0.027578,-0.054397,0.013638,...,0.048271,0.025358,0.005214,-0.036159,0.002611,0.012657,0.086770,-0.001432,0.077808,0.074994
508,noun_sentence_corrections_embeddings_508,-0.039594,-0.063173,-0.037985,0.003814,-0.027960,-0.060582,-0.021174,0.006788,-0.051927,...,0.065591,-0.070694,-0.061728,-0.035065,-0.051671,0.046255,-0.001477,0.059121,0.043726,-0.025050
509,noun_sentence_corrections_embeddings_509,-0.025802,0.000080,-0.023704,0.000022,-0.020402,0.008949,-0.062554,-0.008160,0.003578,...,0.007999,-0.009400,-0.058871,-0.020570,0.014506,0.039482,-0.013598,-0.054112,0.029706,-0.006736
510,noun_sentence_corrections_embeddings_510,0.035652,0.047462,0.079927,0.081814,0.051779,0.026118,0.015016,0.034237,0.026747,...,0.061261,0.028953,-0.027736,0.003590,0.011580,0.028458,-0.020192,0.025230,0.076158,-0.041307


In [56]:
#correlations_merged_df_subset = correlations_merged_df.loc[:,list(noun_sentence_df_50_1000_corr.columns)]

In [57]:
#correlations_merged_df_subset = correlations_merged_df_subset.loc[list(noun_sentence_df_1000_corr.columns)]

In [58]:
len(list(noun_sentence_df_1000_corr.columns))

639

In [59]:
noun_sentence_df_1000_corr

noun_sentence_corrections,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new cabinets,new windows,new driveway,subway tile backsplash,...,great value,short sale foreclosure,large home,private remarks,great opportunity,quick sale,short sale,great investment property,great investment opportunity,needs tender loving care
0,-0.011924,-0.058580,-0.030976,-0.055551,-0.002790,-0.062026,-0.013443,-0.009699,-0.004411,0.012699,...,-0.019999,-0.068759,-0.057489,-0.050721,0.032697,-0.074253,-0.077486,0.055393,0.066990,0.003912
1,-0.040926,-0.078841,0.010525,-0.050620,0.035919,-0.043769,-0.016529,-0.045838,-0.018443,-0.003868,...,-0.094221,-0.071003,-0.017712,0.071219,-0.087883,-0.075865,-0.074032,-0.065019,-0.087347,-0.042235
2,-0.060033,0.007744,0.008026,0.022666,-0.032957,-0.008107,-0.031754,-0.020702,-0.017115,-0.023148,...,-0.064668,0.053975,-0.036073,0.017914,-0.078332,0.018565,0.066595,0.002248,-0.073892,-0.019112
3,-0.022761,-0.043522,-0.024324,0.040247,-0.048845,-0.042315,-0.038782,-0.007243,-0.010116,-0.003280,...,0.004551,-0.039184,-0.001802,-0.030973,-0.022118,0.066049,-0.020082,-0.037914,-0.021683,0.034365
4,-0.073656,-0.080803,0.003008,0.026183,-0.047378,-0.052893,-0.071612,-0.046792,-0.033687,0.023922,...,0.069442,0.055099,0.065950,-0.041265,0.059580,0.017042,0.041439,0.055723,0.045020,0.024409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.025877,0.024143,0.058436,-0.038414,0.036774,0.033655,0.043721,0.056801,0.035004,0.074835,...,-0.026022,0.067697,-0.039932,0.023846,0.006058,-0.041592,0.027428,0.081568,0.078106,0.013642
508,-0.023413,-0.044901,0.056435,-0.005224,-0.049861,-0.027888,-0.038394,-0.033330,-0.015601,-0.050905,...,-0.025732,-0.037336,-0.016188,0.026418,-0.039742,-0.084680,-0.043622,0.043043,0.007136,0.064991
509,-0.012487,-0.042204,-0.035219,-0.043338,0.025568,-0.017498,0.041377,0.000409,-0.005241,0.020724,...,-0.047997,-0.042702,-0.037079,0.033496,0.000353,-0.012300,-0.009963,-0.009728,-0.022563,-0.017943
510,0.038683,0.076957,0.034700,-0.012072,0.076711,0.066418,0.076612,0.039703,0.007052,0.031845,...,-0.031651,0.026585,0.030498,-0.017686,0.000578,-0.002827,0.015390,-0.011720,-0.029145,-0.043296


In [60]:
noun_sentence_df_1000_reindex = noun_sentence_df_1000.set_index('noun_sentence_corrections')

In [61]:
columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
noun_sentence_df_1000_reindex_df = pd.DataFrame(columns=columns)

In [62]:
noun_sentence_df_1000_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings


In [63]:

for key in skill_group:
    #print(key)
    #print(skill_group[key])
    if len(skill_group[key])==1:
        #print(noun_sentence_df_1000_reindex.loc[key,columns])
        noun_sentence_df_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(\
                                                    noun_sentence_df_1000_reindex.loc[key,columns])
        
    else:
        print(skill_group[key])
        temp_df = pd.DataFrame(columns=columns)
        temp_df =  temp_df.append(noun_sentence_df_1000_reindex.loc[skill_group[key],columns].iloc[0])
        for ii in range(3):
            temp_df[noun_sentence_df_1000_reindex.loc[skill_group[key],columns].columns[ii]] = \
                                       [np.concatenate(noun_sentence_df_1000_reindex.loc[skill_group[key],columns]\
                                           [noun_sentence_df_1000_reindex.loc[skill_group[key],columns].columns[ii]].values)]
        noun_sentence_df_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(temp_df.loc[:,columns])
        subgroup = skill_group[key][1:]

['new kitchen cabinets', 'new cabinets', 'kitchen cabinets']
['new kitchen', 'new kitchen appliances']
['new stainless steel appliances', 'new stainless steel appliances']
['new hvac', 'new hvac system']
['custom cabinets', 'custom cabinetry']
['new stainless appliances', 'stainless steel appliances', 'stainless steel appliances', 'stainless steel appliances', 'stainless steel appliances', 'stainless appliances', 'stainless steel appliances']
['new granite counter tops', 'new granite countertops', 'new granite', 'granite counter tops', 'granite countertops', 'granite countertops', 'new granite counters', 'kitchen granite counter tops']
['new hardwood', 'new hardwood floors', 'new hardwoods']
['original hardwood floors', 'hardwood floors', 'hardwood floors', 'hardwood floors', 'hardwood floors']
['new interior exterior paint', 'new exterior paint', 'new interior paint']
['entire home', 'entire house']
['number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'numb

In [64]:
noun_sentence_df_1000_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
new bathrooms,"[0.10878560975609752, 0.0656260194174757, 0.25...","[0.024878048780487806, 0.032362459546925564, -...","[229017180, 229329212, 215966190, 231445008, 1...","[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[0.2046909923664122, 0.25085974999999994, 0.31...","[-0.029770992366412213, -0.07416666666666667, ...","[226851936, 227207328, 228059944, 228694545, 2...","[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[0.07266934782608703, 0.04711003921568624, 0.2...","[0.0, 0.021568627450980392, 0.0, -0.1217073170...","[227301599, 227066289, 227003853, 228078778, 2...","[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.17483815668202762, 0.14467078688524593, 0....","[-0.03686635944700461, -0.04885245901639344, -...","[226661263, 227553043, 227207328, 227436501, 2...","[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[0.3935276169265033, -0.02473273949579835, 0.2...","[0.16481069042316257, -0.008067226890756302, -...","[226027366, 227018382, 228973883, 229099586, 2...","[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...,...,...
great opportunity,"[-0.1604590204081633, -0.0005820000000000825, ...","[-0.02, 0.027777777777777776, -0.0365384615384...","[226851881, 227004045, 227445008, 227386084, 2...","[0.0326966, -0.08788299, -0.07833207, -0.02211..."
quick sale,"[-0.060953463203463176, -0.19054411111111103, ...","[0.0, -0.09722222222222222, -0.1245, -0.005705...","[227579880, 229292063, 230289496, 197222854, 7...","[-0.07425289, -0.07586487, 0.018565405, 0.0660..."
great investment property,"[-0.14633090909090907, 0.15544174757281548, 0....","[-0.13636363636363635, -0.3572815533980582, -0...","[226956668, 212804363, 228971316, 229746211, 2...","[0.055392615, -0.065018825, 0.0022482187, -0.0..."
great investment opportunity,"[-0.05922309803921566, -0.029173655462184866, ...","[-0.05843137254901961, -0.09243697478991597, 0...","[227012367, 227275980, 228799351, 228295649, 2...","[0.0669903, -0.08734736, -0.073891625, -0.0216..."


In [65]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_reindex_df[['noun_sentence_corrections_embeddings']]

In [66]:
noun_sentence_df_1000_corr 

,noun_sentence_corrections_embeddings
new bathrooms,"[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...
great opportunity,"[0.0326966, -0.08788299, -0.07833207, -0.02211..."
quick sale,"[-0.07425289, -0.07586487, 0.018565405, 0.0660..."
great investment property,"[0.055392615, -0.065018825, 0.0022482187, -0.0..."
great investment opportunity,"[0.0669903, -0.08734736, -0.073891625, -0.0216..."


In [67]:
column = 'noun_sentence_corrections_embeddings'
nelements = (len(noun_sentence_df_1000_corr[column][0]))
#print(nelements)
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
#print(column_names)
split_df = pd.DataFrame(noun_sentence_df_1000_corr[column].tolist(), columns=column_names)


In [68]:
split_df

,noun_sentence_corrections_embeddings_0,noun_sentence_corrections_embeddings_1,noun_sentence_corrections_embeddings_2,noun_sentence_corrections_embeddings_3,noun_sentence_corrections_embeddings_4,noun_sentence_corrections_embeddings_5,noun_sentence_corrections_embeddings_6,noun_sentence_corrections_embeddings_7,noun_sentence_corrections_embeddings_8,noun_sentence_corrections_embeddings_9,...,noun_sentence_corrections_embeddings_502,noun_sentence_corrections_embeddings_503,noun_sentence_corrections_embeddings_504,noun_sentence_corrections_embeddings_505,noun_sentence_corrections_embeddings_506,noun_sentence_corrections_embeddings_507,noun_sentence_corrections_embeddings_508,noun_sentence_corrections_embeddings_509,noun_sentence_corrections_embeddings_510,noun_sentence_corrections_embeddings_511
0,-0.011924,-0.040926,-0.060033,-0.022761,-0.073656,0.081821,-0.005458,0.006407,-0.059856,-0.043086,...,-0.001005,-0.076426,0.062387,0.015463,0.041506,0.025877,-0.023413,-0.012487,0.038683,0.034699
1,-0.058580,-0.078841,0.007744,-0.043522,-0.080803,0.049983,0.030043,0.046446,-0.031561,-0.043473,...,0.014715,-0.103510,0.068328,0.001586,0.077301,0.024143,-0.044901,-0.042204,0.076957,-0.000690
2,-0.030976,0.010525,0.008026,-0.024324,0.003008,-0.040600,-0.004780,0.043914,-0.019643,0.012021,...,0.054266,-0.063503,0.074849,0.038589,-0.002148,0.058436,0.056435,-0.035219,0.034700,-0.030047
3,-0.055551,-0.050620,0.022666,0.040247,0.026183,-0.005721,0.053899,0.063238,-0.036279,0.046826,...,0.010186,-0.089421,-0.010154,-0.006709,0.056422,-0.038414,-0.005224,-0.043338,-0.012072,-0.013668
4,-0.002790,0.035919,-0.032957,-0.048845,-0.047378,0.038537,0.014815,0.012746,-0.070697,-0.063335,...,0.043098,-0.059954,0.015364,0.067849,0.037710,0.036774,-0.049861,0.025568,0.076711,0.014323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,0.032697,-0.087883,-0.078332,-0.022118,0.059580,0.016500,-0.011444,0.002790,0.048369,-0.030353,...,-0.054498,-0.097499,0.046397,0.086680,0.054533,0.006058,-0.039742,0.000353,0.000578,-0.051602
452,-0.074253,-0.075865,0.018565,0.066049,0.017042,0.050962,0.057620,-0.029769,0.011815,-0.070450,...,0.049589,-0.093314,0.058984,0.062574,-0.003633,-0.041592,-0.084680,-0.012300,-0.002827,0.009741
453,0.055393,-0.065019,0.002248,-0.037914,0.055723,0.012557,0.047077,-0.013009,-0.016286,-0.061209,...,-0.005474,-0.048618,0.027036,0.088642,-0.062457,0.081568,0.043043,-0.009728,-0.011720,0.019987
454,0.066990,-0.087347,-0.073892,-0.021683,0.045020,-0.000418,0.003835,-0.009434,0.002560,-0.048241,...,-0.033711,-0.078678,0.006397,0.080112,-0.026150,0.078106,0.007136,-0.022563,-0.029145,-0.029067


In [69]:
noun_sentence_df_1000_corr = pd.concat([noun_sentence_df_1000_corr.reset_index(), split_df], axis=1)
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns = [column]).set_index('index')

In [70]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.T

In [71]:
noun_sentence_df_1000_corr

index,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new windows,new driveway,subway tile backsplash,new toilets,...,great lot,great value,short sale foreclosure,large home,private remarks,great opportunity,quick sale,great investment property,great investment opportunity,needs tender loving care
noun_sentence_corrections_embeddings_0,-0.011924,-0.058580,-0.030976,-0.055551,-0.002790,-0.062026,-0.009699,-0.004411,0.012699,-0.022570,...,-0.033516,-0.019999,-0.068759,-0.057489,-0.050721,0.032697,-0.074253,0.055393,0.066990,0.003912
noun_sentence_corrections_embeddings_1,-0.040926,-0.078841,0.010525,-0.050620,0.035919,-0.043769,-0.045838,-0.018443,-0.003868,-0.026864,...,-0.058640,-0.094221,-0.071003,-0.017712,0.071219,-0.087883,-0.075865,-0.065019,-0.087347,-0.042235
noun_sentence_corrections_embeddings_2,-0.060033,0.007744,0.008026,0.022666,-0.032957,-0.008107,-0.020702,-0.017115,-0.023148,-0.065482,...,-0.017070,-0.064668,0.053975,-0.036073,0.017914,-0.078332,0.018565,0.002248,-0.073892,-0.019112
noun_sentence_corrections_embeddings_3,-0.022761,-0.043522,-0.024324,0.040247,-0.048845,-0.042315,-0.007243,-0.010116,-0.003280,-0.008533,...,-0.020068,0.004551,-0.039184,-0.001802,-0.030973,-0.022118,0.066049,-0.037914,-0.021683,0.034365
noun_sentence_corrections_embeddings_4,-0.073656,-0.080803,0.003008,0.026183,-0.047378,-0.052893,-0.046792,-0.033687,0.023922,-0.062892,...,0.072427,0.069442,0.055099,0.065950,-0.041265,0.059580,0.017042,0.055723,0.045020,0.024409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
noun_sentence_corrections_embeddings_507,0.025877,0.024143,0.058436,-0.038414,0.036774,0.033655,0.056801,0.035004,0.074835,-0.019748,...,-0.051012,-0.026022,0.067697,-0.039932,0.023846,0.006058,-0.041592,0.081568,0.078106,0.013642
noun_sentence_corrections_embeddings_508,-0.023413,-0.044901,0.056435,-0.005224,-0.049861,-0.027888,-0.033330,-0.015601,-0.050905,-0.017469,...,-0.021816,-0.025732,-0.037336,-0.016188,0.026418,-0.039742,-0.084680,0.043043,0.007136,0.064991
noun_sentence_corrections_embeddings_509,-0.012487,-0.042204,-0.035219,-0.043338,0.025568,-0.017498,0.000409,-0.005241,0.020724,-0.037228,...,-0.034995,-0.047997,-0.042702,-0.037079,0.033496,0.000353,-0.012300,-0.009728,-0.022563,-0.017943
noun_sentence_corrections_embeddings_510,0.038683,0.076957,0.034700,-0.012072,0.076711,0.066418,0.039703,0.007052,0.031845,0.031121,...,-0.011830,-0.031651,0.026585,0.030498,-0.017686,0.000578,-0.002827,-0.011720,-0.029145,-0.043296


In [72]:
noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index().drop(columns=['index'])

In [73]:
noun_sentence_df_1000_corr

index,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new windows,new driveway,subway tile backsplash,new toilets,...,great lot,great value,short sale foreclosure,large home,private remarks,great opportunity,quick sale,great investment property,great investment opportunity,needs tender loving care
0,-0.011924,-0.058580,-0.030976,-0.055551,-0.002790,-0.062026,-0.009699,-0.004411,0.012699,-0.022570,...,-0.033516,-0.019999,-0.068759,-0.057489,-0.050721,0.032697,-0.074253,0.055393,0.066990,0.003912
1,-0.040926,-0.078841,0.010525,-0.050620,0.035919,-0.043769,-0.045838,-0.018443,-0.003868,-0.026864,...,-0.058640,-0.094221,-0.071003,-0.017712,0.071219,-0.087883,-0.075865,-0.065019,-0.087347,-0.042235
2,-0.060033,0.007744,0.008026,0.022666,-0.032957,-0.008107,-0.020702,-0.017115,-0.023148,-0.065482,...,-0.017070,-0.064668,0.053975,-0.036073,0.017914,-0.078332,0.018565,0.002248,-0.073892,-0.019112
3,-0.022761,-0.043522,-0.024324,0.040247,-0.048845,-0.042315,-0.007243,-0.010116,-0.003280,-0.008533,...,-0.020068,0.004551,-0.039184,-0.001802,-0.030973,-0.022118,0.066049,-0.037914,-0.021683,0.034365
4,-0.073656,-0.080803,0.003008,0.026183,-0.047378,-0.052893,-0.046792,-0.033687,0.023922,-0.062892,...,0.072427,0.069442,0.055099,0.065950,-0.041265,0.059580,0.017042,0.055723,0.045020,0.024409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.025877,0.024143,0.058436,-0.038414,0.036774,0.033655,0.056801,0.035004,0.074835,-0.019748,...,-0.051012,-0.026022,0.067697,-0.039932,0.023846,0.006058,-0.041592,0.081568,0.078106,0.013642
508,-0.023413,-0.044901,0.056435,-0.005224,-0.049861,-0.027888,-0.033330,-0.015601,-0.050905,-0.017469,...,-0.021816,-0.025732,-0.037336,-0.016188,0.026418,-0.039742,-0.084680,0.043043,0.007136,0.064991
509,-0.012487,-0.042204,-0.035219,-0.043338,0.025568,-0.017498,0.000409,-0.005241,0.020724,-0.037228,...,-0.034995,-0.047997,-0.042702,-0.037079,0.033496,0.000353,-0.012300,-0.009728,-0.022563,-0.017943
510,0.038683,0.076957,0.034700,-0.012072,0.076711,0.066418,0.039703,0.007052,0.031845,0.031121,...,-0.011830,-0.031651,0.026585,0.030498,-0.017686,0.000578,-0.002827,-0.011720,-0.029145,-0.043296


In [74]:
noun_sentence_df_50_1000_corr

noun_sentence_corrections,index,luxurious baths,new shaker cabinets,chefs kitchen w stainless steel appliances,tack room,new drywall,white shaker cabinets,perkerson park,new hall bath,stall barn,...,clean property,great fixer,cash sale,digitdigitdigitdigitdigitdigitdigitdigitdigit,excellent opportunity smart bargain hunter,seller requests buyer use seller,great home investors,needs repairs,seller chooses attorney,utilities inspections
0,noun_sentence_corrections_embeddings_0,0.011624,0.032202,-0.002127,-0.045278,-0.000855,0.027027,0.007650,-0.047220,0.034345,...,-0.041365,0.010111,-0.067054,0.060710,0.043231,-0.071548,-0.021866,-0.043543,-0.058099,-0.011068
1,noun_sentence_corrections_embeddings_1,-0.013884,0.042373,-0.015930,0.074813,0.043069,0.071275,0.051528,0.022811,0.044923,...,-0.043555,-0.015113,-0.054921,-0.026173,-0.085849,-0.007990,-0.057591,-0.037721,-0.012980,-0.002281
2,noun_sentence_corrections_embeddings_2,0.068724,-0.002596,0.048211,0.001029,-0.009550,0.009335,0.049954,-0.021367,-0.039344,...,0.041377,-0.054446,0.041126,0.025103,-0.075059,0.044830,-0.037235,0.010220,-0.021597,0.023981
3,noun_sentence_corrections_embeddings_3,0.002632,-0.037502,-0.029201,0.033538,-0.034985,0.018112,0.022226,-0.041952,-0.008421,...,-0.023800,0.019810,0.010891,0.079969,-0.011753,0.017913,-0.001225,0.044749,0.037381,-0.018314
4,noun_sentence_corrections_embeddings_4,0.019046,-0.024633,0.074552,0.051842,-0.025922,0.060065,0.000760,-0.031940,0.066635,...,0.056672,0.070721,0.039114,0.014042,-0.036611,0.083349,0.006665,0.024704,0.016936,-0.066819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,noun_sentence_corrections_embeddings_507,0.010361,0.040626,0.007940,-0.078584,0.045266,0.025759,-0.027578,-0.054397,0.013638,...,0.048271,0.025358,0.005214,-0.036159,0.002611,0.012657,0.086770,-0.001432,0.077808,0.074994
508,noun_sentence_corrections_embeddings_508,-0.039594,-0.063173,-0.037985,0.003814,-0.027960,-0.060582,-0.021174,0.006788,-0.051927,...,0.065591,-0.070694,-0.061728,-0.035065,-0.051671,0.046255,-0.001477,0.059121,0.043726,-0.025050
509,noun_sentence_corrections_embeddings_509,-0.025802,0.000080,-0.023704,0.000022,-0.020402,0.008949,-0.062554,-0.008160,0.003578,...,0.007999,-0.009400,-0.058871,-0.020570,0.014506,0.039482,-0.013598,-0.054112,0.029706,-0.006736
510,noun_sentence_corrections_embeddings_510,0.035652,0.047462,0.079927,0.081814,0.051779,0.026118,0.015016,0.034237,0.026747,...,0.061261,0.028953,-0.027736,0.003590,0.011580,0.028458,-0.020192,0.025230,0.076158,-0.041307


In [75]:
#noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.reset_index().drop(columns=['index','level_0'])

In [76]:
noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.rename(columns={'noun_sentence_corrections':'index'})

In [77]:
noun_sentence_df_50_1000_corr.index.rename('index')

RangeIndex(start=0, stop=512, step=1, name='index')

In [78]:
noun_sentence_df_50_1000_corr

noun_sentence_corrections,index,luxurious baths,new shaker cabinets,chefs kitchen w stainless steel appliances,tack room,new drywall,white shaker cabinets,perkerson park,new hall bath,stall barn,...,clean property,great fixer,cash sale,digitdigitdigitdigitdigitdigitdigitdigitdigit,excellent opportunity smart bargain hunter,seller requests buyer use seller,great home investors,needs repairs,seller chooses attorney,utilities inspections
0,noun_sentence_corrections_embeddings_0,0.011624,0.032202,-0.002127,-0.045278,-0.000855,0.027027,0.007650,-0.047220,0.034345,...,-0.041365,0.010111,-0.067054,0.060710,0.043231,-0.071548,-0.021866,-0.043543,-0.058099,-0.011068
1,noun_sentence_corrections_embeddings_1,-0.013884,0.042373,-0.015930,0.074813,0.043069,0.071275,0.051528,0.022811,0.044923,...,-0.043555,-0.015113,-0.054921,-0.026173,-0.085849,-0.007990,-0.057591,-0.037721,-0.012980,-0.002281
2,noun_sentence_corrections_embeddings_2,0.068724,-0.002596,0.048211,0.001029,-0.009550,0.009335,0.049954,-0.021367,-0.039344,...,0.041377,-0.054446,0.041126,0.025103,-0.075059,0.044830,-0.037235,0.010220,-0.021597,0.023981
3,noun_sentence_corrections_embeddings_3,0.002632,-0.037502,-0.029201,0.033538,-0.034985,0.018112,0.022226,-0.041952,-0.008421,...,-0.023800,0.019810,0.010891,0.079969,-0.011753,0.017913,-0.001225,0.044749,0.037381,-0.018314
4,noun_sentence_corrections_embeddings_4,0.019046,-0.024633,0.074552,0.051842,-0.025922,0.060065,0.000760,-0.031940,0.066635,...,0.056672,0.070721,0.039114,0.014042,-0.036611,0.083349,0.006665,0.024704,0.016936,-0.066819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,noun_sentence_corrections_embeddings_507,0.010361,0.040626,0.007940,-0.078584,0.045266,0.025759,-0.027578,-0.054397,0.013638,...,0.048271,0.025358,0.005214,-0.036159,0.002611,0.012657,0.086770,-0.001432,0.077808,0.074994
508,noun_sentence_corrections_embeddings_508,-0.039594,-0.063173,-0.037985,0.003814,-0.027960,-0.060582,-0.021174,0.006788,-0.051927,...,0.065591,-0.070694,-0.061728,-0.035065,-0.051671,0.046255,-0.001477,0.059121,0.043726,-0.025050
509,noun_sentence_corrections_embeddings_509,-0.025802,0.000080,-0.023704,0.000022,-0.020402,0.008949,-0.062554,-0.008160,0.003578,...,0.007999,-0.009400,-0.058871,-0.020570,0.014506,0.039482,-0.013598,-0.054112,0.029706,-0.006736
510,noun_sentence_corrections_embeddings_510,0.035652,0.047462,0.079927,0.081814,0.051779,0.026118,0.015016,0.034237,0.026747,...,0.061261,0.028953,-0.027736,0.003590,0.011580,0.028458,-0.020192,0.025230,0.076158,-0.041307


In [79]:
correlations_50_1000_df = noun_sentence_df_50_1000_corr.corr()

In [80]:
skill_group_50_1000 = {}
test_df = correlations_50_1000_df.copy('deep')

while len(test_df) > 0:
    idd = test_df.index[0]
    print(idd)
    print(len(test_df))
    group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
    print(len(group))
    skill_group_50_1000[group[0]] = group
    test_df = test_df.drop(labels = group, axis = 1)
    test_df = test_df.drop(labels = group, axis = 0)

luxurious baths
10639
3
new shaker cabinets
10636
3
chefs kitchen w stainless steel appliances
10633
19
tack room
10614
1
new drywall
10613
2
white shaker cabinets
10611
2
perkerson park
10609
12
new hall bath
10597
1
stall barn
10596
2
amazing renovation
10594
12
monday night garage
10582
1
minutes beltline
10581
1
digitstall
10580
1
contemporary finishes
10579
1
modern farmhouse
10578
1
package tile baths
10577
1
new master suite
10576
5
foyer home 
10571
2
modern renovation
10569
1
custom barn doors
10568
3
quartz countertops
10565
8
new white cabinets
10557
1
new systems
10556
1
mercedes benz stadium
10555
1
new white kitchen
10554
1
new kitchen bathrooms
10553
2
omission and or
10551
1
modern design
10550
1
amp service
10549
1
high end stainless steel appliances 
10548
9
custom vent hood
10539
1
top bottom renovation
10538
1
doors drawers
10537
1
designer fixtures
10536
1
water dock
10535
1
new top bottom
10534
1
beverage center
10533
1
white development
10532
1
new quartz tops 
1

private dock
9757
1
new items
9756
1
number foot
9755
2
new custom
9753
2
granite fireplace
9751
1
city convenience
9750
1
spacious bedrooms 
9749
25
new fireplace
9724
1
custom kitchen cabinets
9723
12
central hallway
9711
2
future pool
9709
1
new front porch
9708
2
family chef
9706
1
new cabs
9705
1
ready new roof
9704
1
salt water
9703
1
heart city
9702
2
professional grade appliances
9700
1
clean lines
9699
1
bath cabinets
9698
2
sure wow
9696
1
new crown
9695
1
porcelain tile
9694
2
modern luxury
9692
1
cedar beams
9691
1
bathrooms kitchen
9690
4
top line stainless appliances
9686
1
fantastic views
9685
9
new pool
9676
2
wall cabinets
9674
1
glass wine
9673
1
area fire pit
9672
5
natural stone
9667
1
kitchenaid appliances
9666
1
builder incentives
9665
2
kitchen with custom cabinetry
9663
1
exceptional quality
9662
1
open living concept
9661
3
small barn
9658
1
entire area  wow
9657
1
whirlpool appliances
9656
2
private oasis backyard
9654
2
roof windows
9652
3
mechanical systems


slate floors
8957
2
light switches
8955
1
full stainless steel appliance package
8954
2
deck level
8952
6
granite countertop
8946
2
kitchen counter tops
8944
6
subzero refrigerator
8938
2
open floor concept
8936
5
entertainers dream home
8931
3
gorgeous outdoor
8928
7
whole house attic fan
8921
2
nice size laundry room
8919
7
crepe myrtles
8912
1
kitchen brand
8911
1
baths granite
8910
5
redone kitchen
8905
1
ready fall
8904
1
custom touches
8903
1
massive granite island
8902
4
double french doors lead
8898
4
ceilings w
8894
1
bath tile
8893
8
new countertops
8885
2
home welcomes
8883
2
new shutters
8881
1
new laminate floors
8880
6
brick cottage
8874
1
breakfast bar island
8873
2
luxurious master bathroom
8871
3
wellstar hospital
8868
2
sunrise sunsets plan day
8866
1
custom finishes
8865
1
glass windows
8864
2
backyard space
8862
2
spa shower
8860
1
separate toilet room
8859
1
wrap around deck
8858
1
burner stove
8857
3
granite cntrtops
8854
2
private driveway
8852
2
brazilian cherry

acres privacy
8168
2
steam shower
8166
1
privacy home
8165
5
bedrooms carpet
8160
1
suite bedrooms
8159
3
step media room
8156
1
room furniture
8155
1
kind custom
8154
1
meticulous home
8153
1
builder pay
8152
1
tile counter tops
8151
3
true inlaw suite
8148
1
hardwood floors plantation shutters
8147
1
moment walk front door
8146
2
pull drawers
8144
1
car garage workshop
8143
2
large wrap
8141
2
creek property
8139
1
master guest bedroom
8138
4
piece crown
8134
1
turner field
8133
1
year round
8132
1
number bedrooms
8131
2
plenty outdoor
8129
3
basketball goal
8126
1
yard firepit
8125
2
bath cape cod
8123
1
quick drive
8122
1
schedule visit
8121
2
luxury upgrades
8119
1
digitdigitdigitsqft
8118
1
perfect large gatherings
8117
1
baths hardwood floors
8116
3
true oasis
8113
1
new hardwoods
8112
1
main custom
8111
1
large carport
8110
1
new liner
8109
1
firepit area
8108
1
new wood
8107
1
canton street
8106
1
brick fireplaces
8105
2
perfect workshop
8103
4
expansive windows
8099
2
conveni

ponce city market
7396
1
new high school
7395
1
extraordinary home
7394
1
new pergola
7393
1
new wall wall carpet
7392
4
backyard garden
7388
4
separate island
7384
1
original details
7383
1
suntrust park
7382
1
dark wood floors
7381
5
baths laundry room
7376
6
tub double vanity
7370
8
backyard building
7362
2
island w breakfast
7360
3
new home holidays
7357
1
open floorplan hardwood floors
7356
6
s suite
7350
1
special features
7349
1
upstairs bathroom
7348
4
upgrades everywhere
7344
1
spacious beds
7343
2
main level owners suite
7341
1
show stopper
7340
1
generous kitchen
7339
1
large bdrm
7338
1
room lots
7337
3
ready use
7334
1
ideal family
7333
1
amazing view
7332
5
pantry 
7327
2
dual vanity sinks
7325
2
perfect bbqs
7323
2
picture windows
7321
1
generous living space
7320
2
bedroom study
7318
3
solid surface tops
7315
2
picturesque setting
7313
1
huge fireplace
7312
9
corp property
7303
1
brick stone
7302
2
garage workshop
7300
6
garage wstorage
7294
8
area pets
7286
1
home wow


...

tile splash
2251
1
additional upstairs bedrooms
2250
1
floor w
2249
1
cozy home
2248
2
closets bedrooms
2246
4
main level office
2242
2
gourmet kitchen breakfast
2240
2
pantry laundry room
2238
1
bonus loft area
2237
2
large bathroom double vanity
2235
1
large sunny
2234
1
separate living room office
2233
2
bonusmedia room
2231
1
level bedroom
2230
1
additional large bedrooms
2229
1
master bath walk closet
2228
4
digitbeddigit
2224
1
maximum privacy
2223
1
utility room
2222
1
spacious corner lot
2221
3
full front porch
2218
1
greatroom wfireplace
2217
1
big creek greenway
2216
2
fayette county
2214
3
digitst digitnd
2211
1
miles avalon
2210
1
collection forsyth
2209
1
noble park
2208
1
secondary bedrooms jack
2207
1
mountain park
2206
1
shower double vanity
2205
3
burner cook top
2202
1
breakfast room overlooks
2201
1
brdigitdigit bathroom
2200
2
secondary bedroom
2198
4
impressive price point
2194
1
perfect lot
2193
1
tons storage space
2192
1
large great room w fireplace
2191
2
bath 

large parties
1745
1
actual property
1744
1
blackburn park
1743
1
great room cathedral
1742
3
work station
1739
1
easy access schools
1738
1
traditional brick
1737
3
actual home construction
1734
1
full wall windows
1733
1
luxurious master retreat
1732
1
swimtennis comm
1731
1
patio basement
1730
1
perfect property
1729
1
swift cantrell park
1728
1
large picture windows
1727
1
appointment agent
1726
2
fire place
1724
2
kitchen tons cabinet space
1722
3
inviting fireplace
1719
1
new hospital
1718
2
spacious master suite trey ceilings
1716
2
large front
1714
1
open fireside
1713
1
large tub
1712
1
cozy ranch
1711
1
bdrm home
1710
1
huge bathroom
1709
2
upstairs loft area
1707
1
time buyers need plenty space
1706
1
large lot w
1705
2
digitbrdigitdigit
1703
1
open floorplan home
1702
1
cabinet space
1701
1
separate tub shower walk closet
1700
3
master suite w trey
1697
1
dryer stay
1696
1
easy drive
1695
1
ideal office
1694
2
large bedrooms bath
1692
2
wrap porch
1690
1
porch patio
1689
1


room gorgeous
1269
1
home community
1268
1
den wfp
1267
1
private owners
1266
1
large kitchen granite counter tops
1265
1
craft area
1264
1
close parks
1263
1
spacious formal living room
1262
2
special touches
1260
2
separate showergarden tub
1258
2
small creek
1256
1
forsyth community
1255
1
brick home culdesac
1254
1
available show
1253
1
separate lr
1252
3
large living room wood
1249
1
quiet backyard
1248
1
area backyard
1247
2
basement exterior entry
1245
2
great intown location
1243
2
fresh paint brand
1241
1
digitbdigitdigitb
1240
1
rural setting
1239
1
great location schools
1238
2
ready spring
1236
1
bedroom den
1235
1
room whardwoods
1234
1
excellent home
1233
2
cars garage
1231
1
open floor plan master
1230
1
original owner home
1229
1
panel cabinets
1228
1
fit entire family
1227
1
open backyard
1226
1
lot storage
1225
1
digitbdrmdigitba
1224
1
estate home
1223
2
kitchen accents
1221
1
optional homeowner association
1220
2
lots hardwoods
1218
2
huge clubhouse
1216
1
open floo

acre culdesac
837
1
room eat in kitchen
836
1
space home
835
1
secondary bedrooms lots closet space
834
1
diamond series
833
1
lassiter high school
832
1
main entrance
831
1
central hvac
830
1
room expansion
829
1
heart johns creek
828
1
full bath loft
827
1
bath brick front home
826
2
generous rooms
824
1
level recreation room
823
1
dryer refrigerator
822
3
interior entry
819
1
great living
818
1
separate living room
817
1
indoor outdoor pools
816
1
large floor plan
815
1
area access
814
1
home room
813
1
dickerson middle
812
1
close schools
811
1
ready occupy
810
1
tee box
809
1
fireplace great room
808
3
story great room
805
1
peachtree citys
804
2
spacious w
802
1
new friends
801
1
gourmet kit with island
800
1
fulton county
799
1
convenient gainesville
798
1
ready pets
797
1
convenient master
796
1
traditional features
795
1
pill hill
794
1
distance community pool
793
1
pickle ball
792
1
digitside
791
1
middle school
790
1
quiet family
789
1
great family neighborhood
788
2
spaciou

1
super location
387
1
awesome lot
386
1
stove refrigerator
385
1
mcdonough square
384
1
lake arrowhead
383
1
rental opportunity
382
1
privacy need
381
1
story retreat
380
1
cozy kitchen
379
1
year americas
378
1
lux bath
377
1
locust grove
376
1
rental restriction
375
2
walkin master closet master bathroom dual sinks
373
2
upstairs with number
371
1
fulton school district
370
1
floor plan subdivision
369
1
big canoe
368
1
upstairs trey
367
1
brick wood
366
1
drastic reduction
365
1
swim tennis lake community
364
1
ready imagination
363
1
heart dunwoody
362
1
main level garage
361
1
story brick front
360
1
agent show
359
1
major renovation
358
1
golden opportunity
357
1
separate tub shower dual sinks
356
1
space money
355
1
perfect starter home investment property
354
2
room adjacent
352
1
huge kit
351
1
great new price
350
1
hall location
349
1
number bath
348
3
personal touch
345
2
home lots
343
1
screened in deck
342
1
spacious secondary bedroom
341
1
tenant place
340
1
original pri

In [81]:
len(skill_group_50_1000)

5929

In [82]:
noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000.set_index('noun_sentence_corrections')

In [83]:
noun_sentence_df_50_1000_reindex

,noun_sentence,count,AVM_Error,AVM_Error_List,Listing_Error,Listing_Error_List,Listingid_List,AVM_Error_Average,Listing_Error_Average,AVM_Error_Indicators,Listing_Error_Indicators,noun_sentence_embeddings,noun_sentence_vaderpolarity,noun_sentence_no_punct,not_words,noun_sentence_corrections_embeddings
noun_sentence_corrections,,,,,,,,,,,,,,,,
luxurious baths,luxurious baths,219,51.990258,"[0.5756445471698113, 0.00427920992656361, 0.43...",-3.300529,"[-0.03773584905660377, -0.012661433274246644, ...","[133192257, 139825278, 126928436, 139527311, 6...",0.237398,-0.015071,"[0.23739843674537542, 0.049026637436494276, -0...","[-0.015070907870343365, 0.0029969126970200966,...","[0.011624178, -0.01388437, 0.068723805, 0.0026...","[0.0, 1.0, 0.0, 0.0]",luxurious baths,,"[0.011624178, -0.01388437, 0.068723805, 0.0026..."
new shaker cabinets,new shaker cabinets,68,14.773771,"[0.013355427936272757, 0.25963513157894735, 0....",-1.864840,"[0.0, -0.05236842105263158, -0.035059760956175...","[130901823, 1348486, 140931921, 209025177, 140...",0.217261,-0.027424,"[0.21726134316789047, 0.022322975143823587, 0....","[-0.02742411728406139, 0.001962718666569063, -...","[0.032202173, 0.042373072, -0.0025964438, -0.0...","[0.0, 1.0, 0.0, 0.0]",new shaker cabinets,,"[0.032202173, 0.042373072, -0.0025964438, -0.0..."
chefs kitchen w stainless steel appliances,chefs kitchen w/ ss appliances,53,11.205992,"[-0.04286175105485235, -0.2995155858310625, 0....",-1.276103,"[-0.05464135021097046, -0.021798365122615803, ...","[139884324, 152910992, 139777176, 149525280, 1...",0.211434,-0.024077,"[0.21143380679998347, 0.05013145161043024, -0....","[-0.02407741126789702, 0.0021028086165373583, ...","[0.0061708447, -0.020866258, -0.0043843132, -0...","[0.0, 1.0, 0.0, 0.0]",chefs kitchen w ss appliances,,"[-0.0021265124, -0.015930299, 0.04821109, -0.0..."
tack room,tack room,126,23.355024,"[0.44872725925925916, 0.4897270173410404, 0.02...",-7.719642,"[0.07425925925925926, 0.0, -0.3043478260869565...","[227233008, 233312147, 178762755, 195952044, 2...",0.185357,-0.061267,"[0.1853573304909296, 0.027150458102203588, 0.0...","[-0.06126700368365339, 0.007402598339766534, -...","[-0.04527767, 0.074812554, 0.0010289665, 0.033...","[0.0, 1.0, 0.0, 0.0]",tack room,,"[-0.04527767, 0.074812554, 0.0010289665, 0.033..."
new drywall,new drywall,63,11.563131,"[0.03752599118942723, 0.14459898181818187, 0.0...",-4.126241,"[0.00881057268722467, 0.05454545454545454, 0.0...","[228995631, 139617794, 126936362, 140249369, 1...",0.183542,-0.065496,"[0.18354175614844367, 0.0555186655269487, -1.0...","[-0.06549589446438585, 0.0060286013696304605, ...","[-0.0008549273, 0.04306897, -0.009549875, -0.0...","[0.0, 1.0, 0.0, 0.0]",new drywall,drywall,"[-0.0008549273, 0.04306897, -0.009549875, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
seller requests buyer use seller,seller requests buyer use seller,68,-26.066730,"[-1.3322289999999999, 0.16222735849056613, -0....",-1.395021,"[-0.25, -0.1320754716981132, 0.042553191489361...","[198263962, 200651017, 209001692, 212786867, 1...",-0.383334,-0.020515,"[-0.38333426451694014, 0.14118332477478868, -0...","[-0.020515015912612608, 0.03274158365671943, -...","[-0.071547754, -0.007990354, 0.044830173, 0.01...","[0.0, 1.0, 0.0, 0.0]",seller requests buyer use seller,,"[-0.071547754, -0.007990354, 0.044830173, 0.01..."
great home investors,great home investors,61,-24.093283,"[0.2702147777777778, -0.23938986666666673, -0....",-5.884290,"[-0.1, -0.19866666666666666, -0.08790322580645...","[197054770, 3232636, 151180267, 130595497, 532...",-0.394972,-0.096464,"[-0.3949718558900011, 0.19840775937258184, -0....","[-0.09646376871182749, 0.021014897596130223, -...","[-0.021865588, -0.05759102, -0.037234746, -0.0...","[0.0, 0.328, 0.672, 0.6249]",great home investors,,"[-0.021865588, -0.05759102, -0.037234746, -0.0..."
needs repairs,needs repairs,92,-37.098473,"[-0.7064920000000001, -0.2677977391304347, -0....",-11.679963,"[-0.0476190

In [84]:
columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
noun_sentence_df_50_1000_reindex_df = pd.DataFrame(columns=columns)

In [85]:

for key in skill_group_50_1000:
    print(key)
    #print(skill_group[key])
    if len(skill_group_50_1000[key])==1:
        #print(noun_sentence_df_1000_reindex.loc[key,columns])
        noun_sentence_df_50_1000_reindex_df = noun_sentence_df_50_1000_reindex_df.append(\
                                                    noun_sentence_df_50_1000_reindex.loc[key,columns])
        
    else:
        #print(skill_group_50_1000[key])
        temp_df = pd.DataFrame(columns=columns)
        temp_df =  temp_df.append(noun_sentence_df_50_1000_reindex.loc[[skill_group_50_1000[key][0]],columns].iloc[[0]])
        for ii in range(3):
            temp_df[noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns].columns[ii]] = \
                                       [np.concatenate(noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns]\
                                           [noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns].columns[ii]].values)]
        noun_sentence_df_50_1000_reindex_df = noun_sentence_df_50_1000_reindex_df.append(temp_df.loc[:,columns])
        subgroup = skill_group_50_1000[key][1:]

luxurious baths
new shaker cabinets
chefs kitchen w stainless steel appliances
tack room
new drywall
white shaker cabinets
perkerson park
new hall bath
stall barn
amazing renovation
monday night garage
minutes beltline
digitstall
contemporary finishes
modern farmhouse
package tile baths
new master suite
foyer home 
modern renovation
custom barn doors
quartz countertops
new white cabinets
new systems
mercedes benz stadium
new white kitchen
new kitchen bathrooms
omission and or
modern design
amp service
high end stainless steel appliances 
custom vent hood
top bottom renovation
doors drawers
designer fixtures
water dock
new top bottom
beverage center
white development
new quartz tops 
softclose cabinets
modern finishes
belt line
new kitchen granite countertops
perfect horses
buyers as soon as possible
horse farm
builder spouse
new windows doors
new stainless steel appliance package
new interior exterior paint 
actual plan
ship lap
shaker style cabinets
new addition
new laundry room
new t

new tile shower
fresh paint in out
gorgeous granite counters
number inch
custom windows
discriminating buyers
quality craftsmanship
winter nights
new wide plank
epoxy garage floor
french door refrigerator
built in refrigerator
steps grant park
original charm
new bath fixtures
family room den
interior upgrades
floors bathrooms
number mumber digit
retail district
wooden beams
new patio
large deck outdoor
ice maker
high end
incredible curb appeal
new pool liner
zoysia grass
vegetable garden
bottle wine cellar
fantastic brick ranch
beautiful bath
private virtual tour
spacious open concept
granite stainless steel appliances
bamboo hardwoods
concrete floor
beautiful ground pool
interior designer
fabulous owners suite
trash compactor
professional appliances
full apartment
claw foot tub
state art kitchen
beautiful living room
granite kitchen w
top line appliances
chefs dream
stone shower
gorgeous master bath
floor plan boasts
area home
high end upgrades
digits
new skylights
travertine tile
gri

master suite w walkin closet
minutes decatur
curb appeal galore
family space
entry doors
hardie plank
blocks park
designer upgrades
chair porches
happy home
warm hardwood floors
perfect combination
travertine shower
large mudroom
new outdoor
master retreat envy
garden areas
perfect extra storage
time outdoors
you will fall
time summer
dishwasher microwave
private sanctuary
unique opportunity
proud call home
granite top
quieter lifestyle
new const
s pantry
new hardwood floors kitchen
extra income
new counter top
clean pin
entire yard
garage doors openers
acres privacy
steam shower
privacy home
bedrooms carpet
suite bedrooms
step media room
room furniture
kind custom
meticulous home
builder pay
tile counter tops
true inlaw suite
hardwood floors plantation shutters
moment walk front door
pull drawers
car garage workshop
large wrap
creek property
master guest bedroom
piece crown
turner field
year round
number bedrooms
plenty outdoor
basketball goal
yard firepit
bath cape cod
quick drive
sc

lot size
hard wood
suite bathroom
porch family room
stone backsplash
front porch overlooks
kitchen you will
room floor
new gorgeous
main upper levels
hardwood tile floors
owners suite trey ceilings
yard outdoor
excellent flow
quaint neighborhood
efficient double pane windows
front doors
prime corner lot
sure fall
open house sunday
air units
close beltline
private land
end home
w upgrades
room custom
story fireplace
heavy crown
large island breakfast
fin terrace level
stainless steel
personal tour
full inlaw suite
digitin
new spa bath
adjacent kitchen
porch relax
walkable neighborhood
international horse park
turnkey home
atlanta hartsfield airport
future home
fabulous custom
room home office
beautiful bathroom
miles trails
kit w
impeccable condition
whole home
entire exterior
bonus room garage
small enclave
media space
spacious deck overlooks
master suite w
digitbrdigitb
complete kitchen
work island
dramatic entry
private rear deck
creek runs
outdoor entertainment
new black appliances


cold nights
basement bonus room
hardwood staircase
abundance windows
peach trees
full baths home
privacy fence backyard
roomy master suite
heards ferry
large storage
beautiful lawn
digitbed digitbath
yard thats
low utilities
master boasts
ready features
plenty room pool
study area
unique property
easy appointment
white cabinets granite counters
brick accents
large open area
open front door
game day porch
ideal inlaw suite
mini kitchen
amazing features
home gym
home youve
terrace levels
spacious garage
backyard w
incentives time
brookfield country club
sure check
quality abounds
beautiful new hardwoods
separate cook top
tons character
new interior
blinds windows
private porch
new hvacs
frieze carpet
open concept kitchen
yard 
main campus
movable island
charming fireplace
fabulous brick home
easy commute airport
level backyard
owner retreat
hdwds thruout
remarkable swim community
tanger outlets
massive walk closet
city water
dunwoody home
kitchen cabinetry
oversize master bedroom
bdrmdig

...

open kitchen area
rear yd
private courtyard
oconee county
ceilings master bedroom
cash flow
decorator colors
dramatic staircase
spacious loft area
entertainment center
open floor plan kitchen
bedroom w
j bath
full bath workshop
separate gas cooktop
area w fireplace
backs green space
master bedroom w
main floor guests convenience
home sweet home
kitchen showcases
easy finish
his hers closet
national park
upstairs master
country club community
private backyard w
time market
perfect summer
laminate floor
minutes stonecrest mall
deck patio
powder springs
large pool
rear yards
ideal storage
digitbeddigitbath digitstory
pretty home
large guest rooms
new park
true bedrooms
tile floors kitchen bathrooms
storm shelter
great size bedrooms
mary lin
gorgeous setting
upscale community
clean whistle
beautiful traditional style home
home total
ready new carpet
charm home
flexible floor plan
level drive
mature landscape
lndry room
ready condition
short distance
spacious master
premium countertops
digi

playground community
main level brand
low homeowner association
office kitchen
brdigit bathroom
bonus area
cute kitchen
beautiful owners suite
beautiful condition
large double vanity
story foyer entrance
costs wpref lender
additional bonus room
ola school district
culdesac level lot
downstairs bedroom
digitbrs
brook run park
mother law suite
basement home
extra large deck
perfect media room
wellkept home
restaurants hospital
area basement
main wfp
friendly home
extra touches
private study
luxury kitchen
kitchen hallway
tennis center
city taxes
with new carpet
nursery office
log cabin
open large family room
open gourmet kitchen
great counter space
new microwave
closets galore
sides hardiplank
acre lake
room view kitchen
spacious master retreat
milton school district
new sod front yard
nice features
national recreation area
private ensuite bath
double culdesac
great play
convenient cdc
low maint
low maintenance vinyl exterior
formal din room
his her vanities
pool tennis court
brkfst room

large playground
computer nook
large upstairs bonus room
master bedroom tray
bay windows
backyard wpatio
hard days work
trees privacy
basement expansion
custom built in bookcases
perfect play
white cabs
room spa
charming family home
beautiful home culdesac
lake lanier community
digitdigitba
endless possibilities
separate shwr
large entertainment room
basement plenty room
private stairs
comfortable bedrooms
pantry view family room
great kids
luxurious owners suite
rear lot
large window
brick home corner lot
duck pond
room seat
breakfast area wbay window
large parties
actual property
blackburn park
great room cathedral
work station
easy access schools
traditional brick
actual home construction
full wall windows
luxurious master retreat
swimtennis comm
patio basement
perfect property
swift cantrell park
large picture windows
appointment agent
fire place
kitchen tons cabinet space
inviting fireplace
new hospital
spacious master suite trey ceilings
large front
open fireside
large tub
cozy r

beautiful cherry cabinets
digit acres
fulton schools
book tour
living room with fire place
number bedroom number bathroom digit
darling home
real estate
bath home culdesac
huge ranch
gourmet eat in kitchen
home tons space
digitpm
spacious plan
natural beauty
large daylight basement
tons room
sac location
location cant
schedule showings
community amenities
minutes pinewood studios
active family
basketball court playground
level house
guest retreat
distance silver comet trail
natural lights
huge loft
sunny breakfast area
large fireside family room
open floorplan hardwoods
sunny family room
main level guest bedroom
exceptional opportunity
fabulous amenities
spacious move in
traditional brick front home
autry mill
level backyard patio
eligible usda
bed home
slab home w
beautiful park
bath craftsman style home
restaurants entertainment
office bonus room
bedroom brick home
office play room
bedrooms wjack
st community
fireplace access
special occasions
great family house
new playground
nation

homes subdivision
kitchen oak cabinets
large kitchen pantry
convenient lake lanier
hvac approximate
forsyth neighborhood
nice kitchen breakfast area
ready w
minutes shops
sidewalk community
number square feet
central school district
laundry room garage
home move in condition
bedroom layout
deck family room
hardwoods family room
amazing price
jackson lake
maintenance vinyl
jefferson park
entertainment size
minutes emory
nice area
cabinets overlooks family room
desirable swim
private space
home sits culdesac
great central location
distance neighborhood amenities
seller pay
agent appoinment
loads space
kitchen island breakfast
large size
fitness area
deluxe bath
homeowner association fee
sty family room
wonderful subdivision
spacious brick home
hi ceilings
location key
social activities
nice community
eat in kitchen pantry
deck breakfast area
multiple pools
community minutes
bedrooms bonus
gwinnett braves
pull attic stairs
new copper
carpet paint
bedrooms home
lot space
bright eat in kitc

In [86]:
noun_sentence_df_50_1000_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
luxurious baths,"[0.5756445471698113, 0.00427920992656361, 0.43...","[-0.03773584905660377, -0.012661433274246644, ...","[133192257, 139825278, 126928436, 139527311, 6...","[0.011624178, -0.01388437, 0.068723805, 0.0026..."
new shaker cabinets,"[0.013355427936272757, 0.25963513157894735, 0....","[0.0, -0.05236842105263158, -0.035059760956175...","[130901823, 1348486, 140931921, 209025177, 140...","[0.032202173, 0.042373072, -0.0025964438, -0.0..."
chefs kitchen w stainless steel appliances,"[-0.04286175105485235, -0.2995155858310625, 0....","[-0.05464135021097046, -0.021798365122615803, ...","[139884324, 152910992, 139777176, 149525280, 1...","[-0.0021265124, -0.015930299, 0.04821109, -0.0..."
tack room,"[0.44872725925925916, 0.4897270173410404, 0.02...","[0.07425925925925926, 0.0, -0.3043478260869565...","[227233008, 233312147, 178762755, 195952044, 2...","[-0.04527767, 0.074812554, 0.0010289665, 0.033..."
new drywall,"[0.03752599118942723, 0.14459898181818187, 0.0...","[0.00881057268722467, 0.05454545454545454, 0.0...","[228995631, 139617794, 126936362, 140249369, 1...","[-0.0008549273, 0.04306897, -0.009549875, -0.0..."
...,...,...,...,...
seller requests buyer use seller,"[-1.3322289999999999, 0.16222735849056613, -0....","[-0.25, -0.1320754716981132, 0.042553191489361...","[198263962, 200651017, 209001692, 212786867, 1...","[-0.071547754, -0.007990354, 0.044830173, 0.01..."
great home investors,"[0.2702147777777778, -0.23938986666666673, -0....","[-0.1, -0.19866666666666666, -0.08790322580645...","[197054770, 3232636, 151180267, 130595497, 532...","[-0.021865588, -0.05759102, -0.037234746, -0.0..."
needs repairs,"[-0.7064920000000001, -0.2677977391304347, -0....","[-0.047619047619047616, -0.08695652173913043, ...","[230618952, 187242734, 196407010, 139519327, 5...","[-0.04354262, -0.037721384, 0.010220117, 0.044..."
seller chooses attorney,"[0.06298180555555555, -0.2739625714285714, -0....","[0.02847222222222222, -0.2571428571428571, -0....","[77911179, 189586243, 180510904, 77958589, 854...","[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [87]:
noun_sentence_df_1000_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
new bathrooms,"[0.10878560975609752, 0.0656260194174757, 0.25...","[0.024878048780487806, 0.032362459546925564, -...","[229017180, 229329212, 215966190, 231445008, 1...","[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[0.2046909923664122, 0.25085974999999994, 0.31...","[-0.029770992366412213, -0.07416666666666667, ...","[226851936, 227207328, 228059944, 228694545, 2...","[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[0.07266934782608703, 0.04711003921568624, 0.2...","[0.0, 0.021568627450980392, 0.0, -0.1217073170...","[227301599, 227066289, 227003853, 228078778, 2...","[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.17483815668202762, 0.14467078688524593, 0....","[-0.03686635944700461, -0.04885245901639344, -...","[226661263, 227553043, 227207328, 227436501, 2...","[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[0.3935276169265033, -0.02473273949579835, 0.2...","[0.16481069042316257, -0.008067226890756302, -...","[226027366, 227018382, 228973883, 229099586, 2...","[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...,...,...
great opportunity,"[-0.1604590204081633, -0.0005820000000000825, ...","[-0.02, 0.027777777777777776, -0.0365384615384...","[226851881, 227004045, 227445008, 227386084, 2...","[0.0326966, -0.08788299, -0.07833207, -0.02211..."
quick sale,"[-0.060953463203463176, -0.19054411111111103, ...","[0.0, -0.09722222222222222, -0.1245, -0.005705...","[227579880, 229292063, 230289496, 197222854, 7...","[-0.07425289, -0.07586487, 0.018565405, 0.0660..."
great investment property,"[-0.14633090909090907, 0.15544174757281548, 0....","[-0.13636363636363635, -0.3572815533980582, -0...","[226956668, 212804363, 228971316, 229746211, 2...","[0.055392615, -0.065018825, 0.0022482187, -0.0..."
great investment opportunity,"[-0.05922309803921566, -0.029173655462184866, ...","[-0.05843137254901961, -0.09243697478991597, 0...","[227012367, 227275980, 228799351, 228295649, 2...","[0.0669903, -0.08734736, -0.073891625, -0.0216..."


In [88]:
noun_sentence_df_full_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(noun_sentence_df_50_1000_reindex_df)

In [89]:
noun_sentence_df_full_1000_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
new bathrooms,"[0.10878560975609752, 0.0656260194174757, 0.25...","[0.024878048780487806, 0.032362459546925564, -...","[229017180, 229329212, 215966190, 231445008, 1...","[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[0.2046909923664122, 0.25085974999999994, 0.31...","[-0.029770992366412213, -0.07416666666666667, ...","[226851936, 227207328, 228059944, 228694545, 2...","[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[0.07266934782608703, 0.04711003921568624, 0.2...","[0.0, 0.021568627450980392, 0.0, -0.1217073170...","[227301599, 227066289, 227003853, 228078778, 2...","[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.17483815668202762, 0.14467078688524593, 0....","[-0.03686635944700461, -0.04885245901639344, -...","[226661263, 227553043, 227207328, 227436501, 2...","[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[0.3935276169265033, -0.02473273949579835, 0.2...","[0.16481069042316257, -0.008067226890756302, -...","[226027366, 227018382, 228973883, 229099586, 2...","[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...,...,...
seller requests buyer use seller,"[-1.3322289999999999, 0.16222735849056613, -0....","[-0.25, -0.1320754716981132, 0.042553191489361...","[198263962, 200651017, 209001692, 212786867, 1...","[-0.071547754, -0.007990354, 0.044830173, 0.01..."
great home investors,"[0.2702147777777778, -0.23938986666666673, -0....","[-0.1, -0.19866666666666666, -0.08790322580645...","[197054770, 3232636, 151180267, 130595497, 532...","[-0.021865588, -0.05759102, -0.037234746, -0.0..."
needs repairs,"[-0.7064920000000001, -0.2677977391304347, -0....","[-0.047619047619047616, -0.08695652173913043, ...","[230618952, 187242734, 196407010, 139519327, 5...","[-0.04354262, -0.037721384, 0.010220117, 0.044..."
seller chooses attorney,"[0.06298180555555555, -0.2739625714285714, -0....","[0.02847222222222222, -0.2571428571428571, -0....","[77911179, 189586243, 180510904, 77958589, 854...","[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [90]:
def mergeDictionary(dict_1, dict_2):
    dict_3 = {**dict_1, **dict_2}
    for key, value in dict_3.items():
        if key in dict_1 and key in dict_2:
            dict_3[key] = value + dict_1[key]
    return dict_3

In [91]:
skill_group_full = mergeDictionary(skill_group, skill_group_50_1000)
print(skill_group_full)

{'new bathrooms': ['new bathrooms'], 'new vanities': ['new vanities'], 'quartz countertops': ['quartz countertops', 'quartz countertop', 'quartz counters', 'kitchen quartz countertops', 'quartz counter tops', 'beautiful quartz countertops', 'white quartz countertops', 'new quartz countertops', 'quartz countertops'], 'top bottom': ['top bottom'], 'new kitchen cabinets': ['new kitchen cabinets', 'new cabinets', 'kitchen cabinets'], 'new kitchen': ['new kitchen', 'new kitchen appliances'], 'new windows': ['new windows'], 'new driveway': ['new driveway'], 'subway tile backsplash': ['subway tile backsplash'], 'new toilets': ['new toilets'], 'new gutters': ['new gutters'], 'new deck': ['new deck'], 'new stainless steel appliances': ['new stainless steel appliances', 'new stainless steel appliances'], 'efficient windows': ['efficient windows'], 'new floors': ['new floors'], 'new hvac': ['new hvac', 'new hvac system'], 'private oasis': ['private oasis'], 'new light fixtures': ['new light fixtu

In [92]:
len(skill_group_full)

6325

In [93]:
len(noun_sentence_df_full_1000_reindex_df)

6385

In [94]:
noun_sentence_df_full_1000_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
new bathrooms,"[0.10878560975609752, 0.0656260194174757, 0.25...","[0.024878048780487806, 0.032362459546925564, -...","[229017180, 229329212, 215966190, 231445008, 1...","[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[0.2046909923664122, 0.25085974999999994, 0.31...","[-0.029770992366412213, -0.07416666666666667, ...","[226851936, 227207328, 228059944, 228694545, 2...","[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[0.07266934782608703, 0.04711003921568624, 0.2...","[0.0, 0.021568627450980392, 0.0, -0.1217073170...","[227301599, 227066289, 227003853, 228078778, 2...","[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.17483815668202762, 0.14467078688524593, 0....","[-0.03686635944700461, -0.04885245901639344, -...","[226661263, 227553043, 227207328, 227436501, 2...","[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[0.3935276169265033, -0.02473273949579835, 0.2...","[0.16481069042316257, -0.008067226890756302, -...","[226027366, 227018382, 228973883, 229099586, 2...","[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...,...,...
seller requests buyer use seller,"[-1.3322289999999999, 0.16222735849056613, -0....","[-0.25, -0.1320754716981132, 0.042553191489361...","[198263962, 200651017, 209001692, 212786867, 1...","[-0.071547754, -0.007990354, 0.044830173, 0.01..."
great home investors,"[0.2702147777777778, -0.23938986666666673, -0....","[-0.1, -0.19866666666666666, -0.08790322580645...","[197054770, 3232636, 151180267, 130595497, 532...","[-0.021865588, -0.05759102, -0.037234746, -0.0..."
needs repairs,"[-0.7064920000000001, -0.2677977391304347, -0....","[-0.047619047619047616, -0.08695652173913043, ...","[230618952, 187242734, 196407010, 139519327, 5...","[-0.04354262, -0.037721384, 0.010220117, 0.044..."
seller chooses attorney,"[0.06298180555555555, -0.2739625714285714, -0....","[0.02847222222222222, -0.2571428571428571, -0....","[77911179, 189586243, 180510904, 77958589, 854...","[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [95]:
noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_reindex_df[['noun_sentence_corrections_embeddings']]

In [96]:
noun_sentence_df_full_1000_corr

,noun_sentence_corrections_embeddings
new bathrooms,"[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...
seller requests buyer use seller,"[-0.071547754, -0.007990354, 0.044830173, 0.01..."
great home investors,"[-0.021865588, -0.05759102, -0.037234746, -0.0..."
needs repairs,"[-0.04354262, -0.037721384, 0.010220117, 0.044..."
seller chooses attorney,"[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [97]:
column = 'noun_sentence_corrections_embeddings'
nelements = (len(noun_sentence_df_full_1000_corr[column][0]))
#print(nelements)
list_tmp = [str(i) for i in range(nelements)] 
column_names = [column + '_' + str(i) for i in list_tmp] 
#print(column_names)
split_df = pd.DataFrame(noun_sentence_df_full_1000_corr[column].tolist(), columns=column_names)


In [98]:
split_df

,noun_sentence_corrections_embeddings_0,noun_sentence_corrections_embeddings_1,noun_sentence_corrections_embeddings_2,noun_sentence_corrections_embeddings_3,noun_sentence_corrections_embeddings_4,noun_sentence_corrections_embeddings_5,noun_sentence_corrections_embeddings_6,noun_sentence_corrections_embeddings_7,noun_sentence_corrections_embeddings_8,noun_sentence_corrections_embeddings_9,...,noun_sentence_corrections_embeddings_502,noun_sentence_corrections_embeddings_503,noun_sentence_corrections_embeddings_504,noun_sentence_corrections_embeddings_505,noun_sentence_corrections_embeddings_506,noun_sentence_corrections_embeddings_507,noun_sentence_corrections_embeddings_508,noun_sentence_corrections_embeddings_509,noun_sentence_corrections_embeddings_510,noun_sentence_corrections_embeddings_511
0,-0.011924,-0.040926,-0.060033,-0.022761,-0.073656,0.081821,-0.005458,0.006407,-0.059856,-0.043086,...,-0.001005,-0.076426,0.062387,0.015463,0.041506,0.025877,-0.023413,-0.012487,0.038683,0.034699
1,-0.058580,-0.078841,0.007744,-0.043522,-0.080803,0.049983,0.030043,0.046446,-0.031561,-0.043473,...,0.014715,-0.103510,0.068328,0.001586,0.077301,0.024143,-0.044901,-0.042204,0.076957,-0.000690
2,-0.030976,0.010525,0.008026,-0.024324,0.003008,-0.040600,-0.004780,0.043914,-0.019643,0.012021,...,0.054266,-0.063503,0.074849,0.038589,-0.002148,0.058436,0.056435,-0.035219,0.034700,-0.030047
3,-0.055551,-0.050620,0.022666,0.040247,0.026183,-0.005721,0.053899,0.063238,-0.036279,0.046826,...,0.010186,-0.089421,-0.010154,-0.006709,0.056422,-0.038414,-0.005224,-0.043338,-0.012072,-0.013668
4,-0.002790,0.035919,-0.032957,-0.048845,-0.047378,0.038537,0.014815,0.012746,-0.070697,-0.063335,...,0.043098,-0.059954,0.015364,0.067849,0.037710,0.036774,-0.049861,0.025568,0.076711,0.014323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6380,-0.071548,-0.007990,0.044830,0.017913,0.083349,-0.045600,0.056955,-0.060445,-0.069979,-0.086626,...,0.043277,-0.044363,0.059579,-0.036826,0.080305,0.012657,0.046255,0.039482,0.028458,-0.035270
6381,-0.021866,-0.057591,-0.037235,-0.001225,0.006665,0.037057,0.030569,-0.053313,-0.065151,-0.066282,...,0.055516,-0.034472,0.080640,0.085126,-0.077353,0.086770,-0.001477,-0.013598,-0.020192,-0.014179
6382,-0.043543,-0.037721,0.010220,0.044749,0.024704,0.038501,-0.027768,0.012042,0.037737,0.028988,...,0.058412,-0.067837,0.051379,0.027381,-0.019979,-0.001432,0.059121,-0.054112,0.025230,-0.001131
6383,-0.058099,-0.012980,-0.021597,0.037381,0.016936,-0.007160,0.053621,-0.014170,0.031248,-0.095731,...,0.032972,-0.083984,-0.017622,0.011074,0.074720,0.077808,0.043726,0.029706,0.076158,-0.031519


In [99]:
noun_sentence_df_full_1000_corr = pd.concat([noun_sentence_df_full_1000_corr.reset_index(), split_df], axis=1)
noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.drop(columns = [column]).set_index('index')

In [100]:
noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.T

In [101]:
noun_sentence_df_full_1000_corr

index,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new windows,new driveway,subway tile backsplash,new toilets,...,clean property,great fixer,cash sale,digitdigitdigitdigitdigitdigitdigitdigitdigit,excellent opportunity smart bargain hunter,seller requests buyer use seller,great home investors,needs repairs,seller chooses attorney,utilities inspections
noun_sentence_corrections_embeddings_0,-0.011924,-0.058580,-0.030976,-0.055551,-0.002790,-0.062026,-0.009699,-0.004411,0.012699,-0.022570,...,-0.041365,0.010111,-0.067054,0.060710,0.043231,-0.071548,-0.021866,-0.043543,-0.058099,-0.011068
noun_sentence_corrections_embeddings_1,-0.040926,-0.078841,0.010525,-0.050620,0.035919,-0.043769,-0.045838,-0.018443,-0.003868,-0.026864,...,-0.043555,-0.015113,-0.054921,-0.026173,-0.085849,-0.007990,-0.057591,-0.037721,-0.012980,-0.002281
noun_sentence_corrections_embeddings_2,-0.060033,0.007744,0.008026,0.022666,-0.032957,-0.008107,-0.020702,-0.017115,-0.023148,-0.065482,...,0.041377,-0.054446,0.041126,0.025103,-0.075059,0.044830,-0.037235,0.010220,-0.021597,0.023981
noun_sentence_corrections_embeddings_3,-0.022761,-0.043522,-0.024324,0.040247,-0.048845,-0.042315,-0.007243,-0.010116,-0.003280,-0.008533,...,-0.023800,0.019810,0.010891,0.079969,-0.011753,0.017913,-0.001225,0.044749,0.037381,-0.018314
noun_sentence_corrections_embeddings_4,-0.073656,-0.080803,0.003008,0.026183,-0.047378,-0.052893,-0.046792,-0.033687,0.023922,-0.062892,...,0.056672,0.070721,0.039114,0.014042,-0.036611,0.083349,0.006665,0.024704,0.016936,-0.066819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
noun_sentence_corrections_embeddings_507,0.025877,0.024143,0.058436,-0.038414,0.036774,0.033655,0.056801,0.035004,0.074835,-0.019748,...,0.048271,0.025358,0.005214,-0.036159,0.002611,0.012657,0.086770,-0.001432,0.077808,0.074994
noun_sentence_corrections_embeddings_508,-0.023413,-0.044901,0.056435,-0.005224,-0.049861,-0.027888,-0.033330,-0.015601,-0.050905,-0.017469,...,0.065591,-0.070694,-0.061728,-0.035065,-0.051671,0.046255,-0.001477,0.059121,0.043726,-0.025050
noun_sentence_corrections_embeddings_509,-0.012487,-0.042204,-0.035219,-0.043338,0.025568,-0.017498,0.000409,-0.005241,0.020724,-0.037228,...,0.007999,-0.009400,-0.058871,-0.020570,0.014506,0.039482,-0.013598,-0.054112,0.029706,-0.006736
noun_sentence_corrections_embeddings_510,0.038683,0.076957,0.034700,-0.012072,0.076711,0.066418,0.039703,0.007052,0.031845,0.031121,...,0.061261,0.028953,-0.027736,0.003590,0.011580,0.028458,-0.020192,0.025230,0.076158,-0.041307


In [102]:
noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.reset_index().drop(columns=['index'])

In [103]:
noun_sentence_df_full_1000_corr

index,new bathrooms,new vanities,quartz countertops,top bottom,new kitchen cabinets,new kitchen,new windows,new driveway,subway tile backsplash,new toilets,...,clean property,great fixer,cash sale,digitdigitdigitdigitdigitdigitdigitdigitdigit,excellent opportunity smart bargain hunter,seller requests buyer use seller,great home investors,needs repairs,seller chooses attorney,utilities inspections
0,-0.011924,-0.058580,-0.030976,-0.055551,-0.002790,-0.062026,-0.009699,-0.004411,0.012699,-0.022570,...,-0.041365,0.010111,-0.067054,0.060710,0.043231,-0.071548,-0.021866,-0.043543,-0.058099,-0.011068
1,-0.040926,-0.078841,0.010525,-0.050620,0.035919,-0.043769,-0.045838,-0.018443,-0.003868,-0.026864,...,-0.043555,-0.015113,-0.054921,-0.026173,-0.085849,-0.007990,-0.057591,-0.037721,-0.012980,-0.002281
2,-0.060033,0.007744,0.008026,0.022666,-0.032957,-0.008107,-0.020702,-0.017115,-0.023148,-0.065482,...,0.041377,-0.054446,0.041126,0.025103,-0.075059,0.044830,-0.037235,0.010220,-0.021597,0.023981
3,-0.022761,-0.043522,-0.024324,0.040247,-0.048845,-0.042315,-0.007243,-0.010116,-0.003280,-0.008533,...,-0.023800,0.019810,0.010891,0.079969,-0.011753,0.017913,-0.001225,0.044749,0.037381,-0.018314
4,-0.073656,-0.080803,0.003008,0.026183,-0.047378,-0.052893,-0.046792,-0.033687,0.023922,-0.062892,...,0.056672,0.070721,0.039114,0.014042,-0.036611,0.083349,0.006665,0.024704,0.016936,-0.066819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.025877,0.024143,0.058436,-0.038414,0.036774,0.033655,0.056801,0.035004,0.074835,-0.019748,...,0.048271,0.025358,0.005214,-0.036159,0.002611,0.012657,0.086770,-0.001432,0.077808,0.074994
508,-0.023413,-0.044901,0.056435,-0.005224,-0.049861,-0.027888,-0.033330,-0.015601,-0.050905,-0.017469,...,0.065591,-0.070694,-0.061728,-0.035065,-0.051671,0.046255,-0.001477,0.059121,0.043726,-0.025050
509,-0.012487,-0.042204,-0.035219,-0.043338,0.025568,-0.017498,0.000409,-0.005241,0.020724,-0.037228,...,0.007999,-0.009400,-0.058871,-0.020570,0.014506,0.039482,-0.013598,-0.054112,0.029706,-0.006736
510,0.038683,0.076957,0.034700,-0.012072,0.076711,0.066418,0.039703,0.007052,0.031845,0.031121,...,0.061261,0.028953,-0.027736,0.003590,0.011580,0.028458,-0.020192,0.025230,0.076158,-0.041307


In [104]:
correlations_full_1000_df = noun_sentence_df_full_1000_corr.corr()

In [105]:
skill_group_full_1000 = {}
test_df = correlations_full_1000_df.copy('deep')

while len(test_df) > 0:
    idd = test_df.index[0]
    print(idd)
    print(len(test_df))
    group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
    print(len(group))
    skill_group_full_1000[group[0]] = group
    test_df = test_df.drop(labels = group, axis = 1)
    test_df = test_df.drop(labels = group, axis = 0)

new bathrooms
6385
3
new vanities
6382
1
quartz countertops
6381
2
top bottom
6379
1
new kitchen cabinets
6378
3
new kitchen
6375
4
new windows
6371
3
new driveway
6368
1
subway tile backsplash
6367
4
new toilets
6363
2
new gutters
6361
3
new deck
6358
4
new stainless steel appliances
6354
7
efficient windows
6347
1
new floors
6346
3
new hvac
6343
2
private oasis
6341
3
new light fixtures
6338
3
custom cabinets
6335
3
new fixtures
6332
2
new granite counter tops
6330
5
new exterior
6325
2
number mumber
6323
1
custom home
6322
4
wine cooler
6318
2
fire pit
6316
2
new hardwood
6314
3
new appliances
6311
2
atlanta village
6309
2
new tile
6307
3
light fixtures
6304
2
original hardwood floors
6302
3
kitchen baths
6299
2
new interior exterior paint
6297
2
tile bathrooms
6295
3
open concept
6292
6
kitchen bathrooms
6286
2
new carpet bedrooms
6284
3
interior exterior
6281
2
entire home
6279
1
wine cellar
6278
2
number
6276
2
new construction
6274
2
hvac system
6272
1
gorgeous hardwood floors
6

large family room fireplace
5211
3
bedroom 
5208
2
traditional home
5206
2
convenient i-number
5204
2
digitst
5202
1
great space
5201
3
large breakfast area
5198
2
digitbr
5196
2
large master bath
5194
3
number bedroom number bathroom
5191
4
tennis courts
5187
3
large eat in kitchen
5184
4
excellent schools
5180
2
level driveway
5178
3
formal lr
5175
2
digitdigitk
5173
2
master bath garden tub
5171
2
number story
5169
2
foyer home
5167
2
convenient location
5165
4
separate living
5161
1
formal living room
5160
4
huge kitchen
5156
2
community pool
5154
2
formal living
5152
2
nice size
5150
3
elementary school
5147
2
brick front
5145
4
swimming tennis community
5141
6
eat in kitchen
5135
4
school district
5131
3
tennis community
5128
1
bedroom ranch
5127
2
separate living room
5125
2
lawn maintenance
5123
5
number sides
5118
2
rear stairs
5116
2
neighborhood amenities
5114
1
huge master bedroom
5113
1
tons space
5112
1
hardwoods foyer
5111
2
home vacant
5109
2
partial basement
5107
2
spa

salt water
4743
1
heart city
4742
1
professional grade appliances
4741
1
clean lines
4740
1
bath cabinets
4739
1
sure wow
4738
1
new crown
4737
1
porcelain tile
4736
1
modern luxury
4735
1
cedar beams
4734
1
top line stainless appliances
4733
1
fantastic views
4732
1
new pool
4731
1
wall cabinets
4730
1
glass wine
4729
1
natural stone
4728
1
kitchenaid appliances
4727
1
builder incentives
4726
1
kitchen with custom cabinetry
4725
1
exceptional quality
4724
1
small barn
4723
1
entire area  wow
4722
1
whirlpool appliances
4721
1
roof windows
4720
1
mechanical systems
4719
1
double pane
4718
1
beautiful creek
4717
1
shower heads
4716
1
marta station
4715
1
covington square
4714
1
fish pond
4713
1
cabinet doors
4712
1
new hardy plank
4711
1
distance smyrna market village
4710
1
new electrical panel
4709
1
portion basement
4708
1
new energy
4707
1
oakhurst village
4706
1
kitchen wall
4705
1
carriage house
4704
1
urban oasis
4703
1
porch access
4702
1
charm galore
4701
1
marble shower
4700
1

stone wall
4378
1
leaf guard gutters
4377
1
spacious hall bath
4376
1
classic style
4375
1
ready new homeowner
4374
1
tile hardwoods
4373
1
perfect reading
4372
1
wood stove
4371
1
with hard wood floors
4370
1
new bamboo floors
4369
1
full lawn maintenance included focus life
4368
1
space w
4367
1
new septic
4366
1
frank lloyd wright
4365
1
new pump
4364
1
double shower
4363
1
bells whistles
4362
1
charming craftsman
4361
1
convection microwave
4360
1
shower surround
4359
1
charming move in
4358
1
lush landscape
4357
1
solid oak cabinets
4356
1
large shop
4355
1
heart brookhaven
4354
1
todays lifestyle
4353
1
smart home
4352
1
separate driveway
4351
1
glass cabinets
4350
1
beautiful country home
4349
1
new cabinets appliances
4348
1
open main floor
4347
1
formal spaces
4346
1
bosch driveway
4345
1
custom closet
4344
1
seamless shower door
4343
1
complete brand
4342
1
rest house
4341
1
spacious office
4340
1
amazing outdoor space
4339
1
pella windows
4338
1
freedom park
4337
1
recess li

functional floor plan
4017
1
two car carport
4016
1
pickiest buyers
4015
1
grill deck
4014
1
digitperson
4013
1
wow factor
4012
1
rich hardwood
4011
1
large family rooms
4010
1
holes golf
4009
1
main floor bedrooms
4008
1
concept floor plan
4007
1
fixtures 
4006
1
stainless steel fridge
4005
2
hartsfield airport
4003
1
kitchen upgrades
4002
1
plan w
4001
1
remote control
4000
1
extra garage
3999
1
excellent area
3998
1
pristine hardwoods
3997
1
owners suite w
3996
1
area owners
3995
1
beauty home
3994
1
digitm
3993
1
buyers fall
3992
1
pine floors
3991
1
garden beds
3990
1
interior boasts
3989
1
hot water tank
3988
1
s dream
3987
1
elegant bath
3986
1
digitdigitacre
3985
1
good book
3984
1
high efficiency
3983
1
spectacular views
3982
1
jackson county
3981
1
brazilian hardwoods
3980
1
room w french doors
3979
1
theater room
3978
1
secret garden
3977
1
master suite w fireplace
3976
1
oversize bedrooms
3975
1
pendant lights
3974
1
space fireplace
3973
1
area w
3972
1
master bath tile sho

...

1
office w
1838
1
french drs
1837
1
couple years
1836
1
entire family
1835
1
spacious features
1834
1
marble bathrooms
1833
1
double door entry
1832
1
large double
1831
1
parks schools
1830
1
generous master
1829
1
desirable johns creek
1828
1
upstairs home
1827
1
end street
1826
1
bonus space
1825
1
separate cooktop
1824
1
cabinets breakfast area
1823
1
convenient interstates
1822
1
chair railing
1821
1
spacious breakfast room
1820
1
story foyer hardwood
1819
1
qualifies usda
1818
1
fresh interior
1817
1
separate sunroom
1816
1
master shower seat glass enclosure
1815
1
bed room
1814
2
king size bed
1812
1
bedroom option
1811
1
easy maintenance
1810
1
proximity i-number
1809
1
  seller
1808
1
nature lovers dream
1807
1
home plenty
1806
1
whirlpool garden tub
1805
1
full bath wet
1804
1
quiet location
1803
1
culdesac location
1802
1
entertainment venues
1801
1
picture frame crown
1800
1
restaurants i-number
1799
1
palladium window
1798
1
lenox mall
1797
1
beautiful front yard
1796
1
luc

additional large bedrooms
1487
1
master bath walk closet
1486
1
digitbeddigit
1485
1
maximum privacy
1484
1
utility room
1483
1
full front porch
1482
1
greatroom wfireplace
1481
1
big creek greenway
1480
1
fayette county
1479
1
digitst digitnd
1478
1
miles avalon
1477
1
collection forsyth
1476
1
noble park
1475
1
secondary bedrooms jack
1474
1
mountain park
1473
1
shower double vanity
1472
1
burner cook top
1471
1
brdigitdigit bathroom
1470
1
impressive price point
1469
1
perfect lot
1468
1
bath family room
1467
1
cherry cabs
1466
1
spare bedrooms
1465
1
secondary bedrooms walkin closets
1464
1
flexible space
1463
1
plenty privacy
1462
1
short drive
1461
1
archer high school district
1460
1
beautiful trey
1459
1
underground utilities
1458
1
brick front beauty
1457
1
gourmet kit
1456
1
lot home
1455
1
eat in area
1454
1
built in book cases
1453
1
grayson school district
1452
1
cobb county schools
1451
1
hot neighborhood
1450
1
property backs
1449
1
popular community
1448
1
lambert schoo

common area
1126
1
airy floor plan
1125
1
roof approximate
1124
1
lake dow
1123
1
wood deck
1122
1
huge living room
1121
1
town center
1120
1
maple cabs
1119
1
priv backyard
1118
1
great access
1117
1
new neutral carpet
1116
1
traditional sale
1115
1
public water
1114
1
small family
1113
1
ashton woods
1112
1
two story family room fireplace
1111
1
brick home master
1110
1
kitchen wgranite ctops
1109
1
beautiful mountain
1108
1
acre yard
1107
1
large garden tub
1106
1
check home
1105
1
counter space
1104
1
recent paint
1103
1
popular master
1102
1
 brand
1101
1
grand entrance
1100
1
location location location
1099
1
central park
1098
1
school system
1097
1
great features home
1096
1
digitbd
1095
1
convenient highway
1094
1
new disposal
1093
1
open den
1092
1
bath ranch bonus room
1091
1
bath bungalow
1090
1
social gatherings
1089
1
top hill
1088
1
shadow box
1087
1
close i-number
1086
2
lot 
1084
1
study wfrench doors
1083
1
georgian plan
1082
1
coffer ceilings
1081
1
large furniture
10

1
number street
760
1
gracious home
759
1
sugarloaf pkwy
758
1
schedule showingtime
757
1
ranchstyle home
756
1
private enclave
755
1
formal lr dinning room
754
1
collins hill school district
753
1
family room with built ins
752
1
popular floorplan
751
1
great location highway
750
1
small community
749
1
lovely view
748
1
room wbay window
747
1
sarah smith school district
746
1
guest brba
745
1
distance top
744
1
roswell address
743
1
split foyer home
742
1
charming features
741
1
new builds
740
1
formal entry
739
1
office study
738
1
legacy park
737
1
open flr pln
736
1
miles golf cart paths
735
1
mill creek school district
734
1
main floorplan
733
1
arbor place mall
732
1
floor study
731
1
condition home
730
1
vanity master
729
1
circle drive
728
1
multiple rooms
727
1
price wont
726
1
brookwood high school
725
1
recent appraisal
724
1
sandy springs
723
1
mandatory homeowner association
722
1
cobb location
721
1
bedroomdigitdigit bathroom
720
1
hardwood entry
719
1
unique open floor 

365
1
great community amenities
364
1
tax credit
363
1
close ga
362
1
ready june
361
1
unbelievable value
360
1
public schools
359
1
carpet allowance
358
1
two story culdesac home
357
1
digitdigithour
356
1
kitchen eat in area
355
1
time home buyers investors
354
1
industrial boulevard
353
1
room master bath
352
1
wonderful family
351
1
huge spa bath
350
1
community pool tennis courts
349
1
private backyard storage
348
1
large open kitchen breakfast area
347
1
large trees
346
1
attractive home
345
1
scenic highway
344
1
gwinnett mall
343
1
floor plan community
342
1
madison plan
341
1
purchase home
340
1
desirable school district
339
1
bonus room master
338
1
cluster home
337
1
fit needs
336
1
need room
335
1
private property
334
1
corporate relocation
333
1
feet home
332
1
private bath garden tub
331
1
sure catch eye
330
1
wonderful traditional home
329
1
emory cdc
328
1
great shape
327
1
open feeling
326
1
quick response
325
1
showings appointment
324
1
visit house
323
1
neutral tone

In [106]:
len((skill_group_full_1000))

5421

In [107]:
len(skill_group_full_1000)

skill_group_full_final = skill_group_full_1000.copy()

for key in skill_group_full_1000:
    list_tmp = skill_group_full[key]
    skill_group_full_final[key] = skill_group_full_1000[key] + list_tmp


In [108]:
skill_group_full_final

{'new bathrooms': ['new bathrooms',
  'new kitchen bathrooms',
  'new vanities bathrooms',
  'new bathrooms'],
 'new vanities': ['new vanities', 'new vanities'],
 'quartz countertops': ['quartz countertops',
  'quartz countertops',
  'quartz countertops',
  'quartz countertop',
  'quartz counters',
  'kitchen quartz countertops',
  'quartz counter tops',
  'beautiful quartz countertops',
  'white quartz countertops',
  'new quartz countertops',
  'quartz countertops'],
 'top bottom': ['top bottom', 'top bottom'],
 'new kitchen cabinets': ['new kitchen cabinets',
  'new cabinets kitchen',
  'custom kitchen cabinets',
  'new kitchen cabinets',
  'new cabinets',
  'kitchen cabinets'],
 'new kitchen': ['new kitchen',
  'new custom kitchen',
  'new kitchen w',
  'new tile kitchen',
  'new kitchen',
  'new kitchen appliances'],
 'new windows': ['new windows',
  'new windows doors',
  'new thermopane windows',
  'new windows'],
 'new driveway': ['new driveway', 'new driveway'],
 'subway tile 

In [109]:
len(skill_group_full_final)

5421

In [110]:
columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
noun_sentence_df_final_reindex_df = pd.DataFrame(columns=columns)

In [111]:

for key in skill_group_full_1000:
    print(key)
    #print(skill_group[key])
    if len(skill_group_full_1000[key])==1:
        #print(noun_sentence_df_1000_reindex.loc[key,columns])
        noun_sentence_df_final_reindex_df = noun_sentence_df_final_reindex_df.append(\
                                                    noun_sentence_df_full_1000_reindex_df.loc[key])
        
    else:
        #print(skill_group_50_1000[key])
        temp_df = pd.DataFrame(columns=columns)
        temp_df =  temp_df.append(noun_sentence_df_full_1000_reindex_df.loc[[skill_group_full_1000[key][0]]].iloc[[0]])
        for ii in range(3):
            temp_df[noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]].columns[ii]] = \
                               [np.concatenate(noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]]\
                               [noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]].columns[ii]].values)]
        noun_sentence_df_final_reindex_df = noun_sentence_df_final_reindex_df.append(temp_df)

new bathrooms
new vanities
quartz countertops
top bottom
new kitchen cabinets
new kitchen
new windows
new driveway
subway tile backsplash
new toilets
new gutters
new deck
new stainless steel appliances
efficient windows
new floors
new hvac
private oasis
new light fixtures
custom cabinets
new fixtures
new granite counter tops
new exterior
number mumber
custom home
wine cooler
fire pit
new hardwood
new appliances
atlanta village
new tile
light fixtures
original hardwood floors
kitchen baths
new interior exterior paint
tile bathrooms
open concept
kitchen bathrooms
new carpet bedrooms
interior exterior
entire home
wine cellar
number
new construction
hvac system
gorgeous hardwood floors
water heater
tile shower
white cabinets
hot tub
gorgeous hardwoods
new roof
mud room
upgrades galore
digitdigitft
garden area
privacy fence
new countertops
digit
huge island
granite counters
walkin shower
new garage doors
exterior paint
inground pool
large shower
beautiful kitchen
curb appeal
chefs kitchen
c

entire property
custom tile work
original heart pine floors
carrera marble
great water
new bathroom fixtures
feet lake frontage
clawfoot tub
solid hardwood
inch home
new full bath
concrete counters
details list
low e windows
new cabinetry
historic charm
old charm
mud area
new concrete driveway
boat dock
lake life
fresh designer paint
pool house
shiplap accents
private estate
thermador appliances
large barn
quiet cove
electrical system
palm trees
top line finishes
new light
krog street market
close drawers
new duct work
frameless glass
pot filler
modern open floor plan
beautiful remodel
bessie branham park
kitchen aid appliances
private park
new interior doors
oak hardwood floors
modern light fixtures
additional acreage
original fireplace
roll doors
foam insulation
inner chef
spectacular custom
westside reservoir park
pole barn
modern flair
incredible lake
capshaw homes
double driveway
views lake
luxurious finishes
new features
custom vanity
designer tile
perfect blend
classic charm
mod

new low flow toilets
true hardwood floors
beautiful bungalow
master bath suite
true chefs kitchen
historic acworth
fine details
bathrooms granite
private living
outdoor firepit
french doors kitchen
ensuite bathroom
sheet rock
cape cod home
dead gorgeous
complete wet
tv area
shower bench
fault seller
number acres
new lvt
new faucets
beautiful country setting
multiple vehicles
attic insulation
family members
inman park
stone wall
leaf guard gutters
spacious hall bath
classic style
ready new homeowner
tile hardwoods
perfect reading
wood stove
with hard wood floors
new bamboo floors
full lawn maintenance included focus life
space w
new septic
frank lloyd wright
new pump
double shower
bells whistles
charming craftsman
convection microwave
shower surround
charming move in
lush landscape
solid oak cabinets
large shop
heart brookhaven
todays lifestyle
smart home
separate driveway
glass cabinets
beautiful country home
new cabinets appliances
open main floor
formal spaces
bosch driveway
custom c

extraordinary home
new pergola
new wall wall carpet
separate island
original details
suntrust park
baths laundry room
backyard building
island w breakfast
new home holidays
s suite
special features
upstairs bathroom
upgrades everywhere
spacious beds
main level owners suite
show stopper
generous kitchen
large bdrm
room lots
ready use
ideal family
amazing view
pantry 
dual vanity sinks
perfect bbqs
picture windows
generous living space
bedroom study
picturesque setting
huge fireplace
corp property
brick stone
garage workshop
garage wstorage
area pets
home wow
moment step
perfect boat
multiple living spaces
unique features
porch wfp
incredible location minutes i-number
new upgrades
built in pantry
local restaurants
private outdoor
acre retreat
open concept designs spaces
lovely foyer
community dock
main with walk in closet
inviting family room
bedsdigit baths
basement with media room
walkin closet master
built in grill
acre setting
paint in out
air ducts
picture window
low cherokee taxes


brookfield country club
sure check
quality abounds
beautiful new hardwoods
separate cook top
tons character
new interior
blinds windows
private porch
frieze carpet
open concept kitchen
yard 
main campus
movable island
charming fireplace
easy commute airport
owner retreat
hdwds thruout
remarkable swim community
tanger outlets
massive walk closet
city water
dunwoody home
kitchen cabinetry
oversize master bedroom
peaceful setting
expansive views
ready ranch home
roof brand
city jefferson schools
young roof
main guests
style home
braves stadium
guest bedroom office
beautiful chandelier
amazing sunroom
home backs
french door deck
lender credit
tons cabinet
invisible fence
home awaits
recreation area
banquet dinning room
open deck
hardwoods levels
main spa
room lead
restaurants parks
perfect entertainment
closets master
inviting family room fireplace
etc 
tub surround
secondary bdrm
game area
ranch home basement
master ensuite
beds 
great room w
marble vanity
custom paint colors
upstairs bas

...

great colors
long drive
huge master w
close airport
fireplace wood
complete level
quiet sd
new carpet stairs
car side entry
spacious media room
medical center
eat in kitchen view family room
miles ga
wow 
extra hardwoods
digitth digitth
double crown
schedule appoinment
water closet
upstairs storage
full bed bath
tons extras
new place
faux wood blinds
neat pin
kitchen double ovens
patio doors
tribble mill park
convenient ga
distance parks
social circle schools
family room wfp
gourmet kitchen w island
rooms downstairs
digitpiece
loads storage
beautiful hand
community pavilion
ceilings kitchen
book cases
separate bathroom
room view
property sits
alpharetta address
spacious front
private ensuite
great room floor
bay window breakfast area
master bedroom fireplace
harrison high
historic square
standup shower
large building
amazing details
great attractions business venues atlanta
wall windows family room
deck stairs
owners suite tray
great care
 move in
distance award
lifetime opportunity
su

great room gas log fireplace
whitewater schools
past fence
yard backs woods
main floor boasts
bedrooms jack
palladian window
school dist
fantastic level lot
family room wet
fit royalty
large master suite double vanities
area garage
beautiful corner lot
convenient idigitdigitdigit
digitdigitxdigitdigitdigit
main tray
edward plan
fowler park
easy commutes
beautiful flowers
lot neighborhood
large brick
digitbeddigitdigit
breakfast areas
car gar
soccer fields
mediagame room
separate den
separate den fireplace
sugarloaf mills
separate bonus room
great swimtennis nhood
room w bay window
fireside master suite
great room custom
public park
property line
digitbedroom
big master
woodland school district
digitndry
complete walkin closet
yard privacy
intercom system
area trey
forsyth taxes
bedrooms basement
inviting home
efficient features
additional closet
new move in
tree line
upper floor
cozy breakfast area
coweta county
office craft room
jr olympic pool
forsyth county schools
extra large garag

large entertainment room
private stairs
comfortable bedrooms
pantry view family room
great kids
luxurious owners suite
rear lot
brick home corner lot
duck pond
breakfast area wbay window
large parties
actual property
blackburn park
great room cathedral
work station
easy access schools
traditional brick
actual home construction
full wall windows
luxurious master retreat
swimtennis comm
patio basement
perfect property
swift cantrell park
appointment agent
fire place
kitchen tons cabinet space
inviting fireplace
new hospital
spacious master suite trey ceilings
large front
open fireside
large tub
cozy ranch
bdrm home
huge bathroom
time buyers need plenty space
large lot w
digitbrdigitdigit
open floorplan home
cabinet space
separate tub shower walk closet
master suite w trey
dryer stay
easy drive
ideal office
large bedrooms bath
porch patio
upstairs office
opportunity home
plenty counter space
permanent stairs attic
photos renderings representative
basement potential
room bonus room
formal 

mulberry park
coat valet
formal den
separate family
quaint home
agent information
additional den
kennesaw state university
separate formal living
ez show
peaceful country
central location
athletic club
timber ridge
john with island resale
chastain park
floor owners suite
 wont
vanity 
view home
nice bedrooms
built in oven
great room double
price home
upstairs carpet
low traffic
neighborhood pool
lots options
expansive great room
nearby restaurants
sugarloaf parkway
floor office
space privacy
formal liv
utility building
beautiful home w
island kit
hard surface counters
perimeter mall
lawn mower
green belts
true showplace
bright sunny
private bath walk closet
story ceilings
home good
interstate i-number
corner garden tub
low cobb county taxes
selections carpet
huge country kitchen
main level trey
basement culdesac
foyer plan
ample cabinet
cozy front porch
dead end street
digitdigithole
with connector
close lake
medical facilities
easy access parks
convenient schools
easy commute emory
bu

upstairs fireplace
great home price
vinyl exterior
lots house
new construction george tomas homes
convenient shops
solid construction
tons square footage
fabulous price
new photos
excellent school system
paint allowance
available quick move in
great house money
lrdr combo
lot room
imagination run
cute house
fast price
huge reduction
owner occupants
size home
great square footage
perfect personal business needs
owner occupant purchasers
digitdigitdigitmo
cool matter port
seller process
nice size rooms
estate property
additional information
unbeatable value
positive cash flow
time homeowner investor
time home
current home
space price
similar plan
time buyer investor
market price
new community
lots possibilities
homes area
price neighborhood
ready quick sale
lots space money
disclosure seller
serious buyers
low price
level basement
taxes reflect senior discount
rental investment
opportunity knocks
month month
rental property
vacant show
downstairs area
home good condition
instant equity
c

In [112]:
len(noun_sentence_df_final_reindex_df)

5421

In [113]:
noun_sentence_df_final_reindex_df

,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
new bathrooms,"[0.10878560975609752, 0.0656260194174757, 0.25...","[0.024878048780487806, 0.032362459546925564, -...","[229017180, 229329212, 215966190, 231445008, 1...","[-0.011923872, -0.040925846, -0.060033374, -0...."
new vanities,"[0.2046909923664122, 0.25085974999999994, 0.31...","[-0.029770992366412213, -0.07416666666666667, ...","[226851936, 227207328, 228059944, 228694545, 2...","[-0.05858002, -0.078841336, 0.007744186, -0.04..."
quartz countertops,"[0.07266934782608703, 0.04711003921568624, 0.2...","[0.0, 0.021568627450980392, 0.0, -0.1217073170...","[227301599, 227066289, 227003853, 228078778, 2...","[-0.030976344, 0.010525043, 0.008025839, -0.02..."
top bottom,"[-0.17483815668202762, 0.14467078688524593, 0....","[-0.03686635944700461, -0.04885245901639344, -...","[226661263, 227553043, 227207328, 227436501, 2...","[-0.055551246, -0.050620284, 0.022666093, 0.04..."
new kitchen cabinets,"[0.3935276169265033, -0.02473273949579835, 0.2...","[0.16481069042316257, -0.008067226890756302, -...","[226027366, 227018382, 228973883, 229099586, 2...","[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...,...,...
seller requests buyer use seller,"[-1.3322289999999999, 0.16222735849056613, -0....","[-0.25, -0.1320754716981132, 0.042553191489361...","[198263962, 200651017, 209001692, 212786867, 1...","[-0.071547754, -0.007990354, 0.044830173, 0.01..."
great home investors,"[0.2702147777777778, -0.23938986666666673, -0....","[-0.1, -0.19866666666666666, -0.08790322580645...","[197054770, 3232636, 151180267, 130595497, 532...","[-0.021865588, -0.05759102, -0.037234746, -0.0..."
needs repairs,"[-0.7064920000000001, -0.2677977391304347, -0....","[-0.047619047619047616, -0.08695652173913043, ...","[230618952, 187242734, 196407010, 139519327, 5...","[-0.04354262, -0.037721384, 0.010220117, 0.044..."
seller chooses attorney,"[0.06298180555555555, -0.2739625714285714, -0....","[0.02847222222222222, -0.2571428571428571, -0....","[77911179, 189586243, 180510904, 77958589, 854...","[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [114]:
noun_sentence_df_final_reindex_df.reset_index()

,index,AVM_Error_List,Listing_Error_List,Listingid_List,noun_sentence_corrections_embeddings
0,new bathrooms,"[0.10878560975609752, 0.0656260194174757, 0.25...","[0.024878048780487806, 0.032362459546925564, -...","[229017180, 229329212, 215966190, 231445008, 1...","[-0.011923872, -0.040925846, -0.060033374, -0...."
1,new vanities,"[0.2046909923664122, 0.25085974999999994, 0.31...","[-0.029770992366412213, -0.07416666666666667, ...","[226851936, 227207328, 228059944, 228694545, 2...","[-0.05858002, -0.078841336, 0.007744186, -0.04..."
2,quartz countertops,"[0.07266934782608703, 0.04711003921568624, 0.2...","[0.0, 0.021568627450980392, 0.0, -0.1217073170...","[227301599, 227066289, 227003853, 228078778, 2...","[-0.030976344, 0.010525043, 0.008025839, -0.02..."
3,top bottom,"[-0.17483815668202762, 0.14467078688524593, 0....","[-0.03686635944700461, -0.04885245901639344, -...","[226661263, 227553043, 227207328, 227436501, 2...","[-0.055551246, -0.050620284, 0.022666093, 0.04..."
4,new kitchen cabinets,"[0.3935276169265033, -0.02473273949579835, 0.2...","[0.16481069042316257, -0.008067226890756302, -...","[226027366, 227018382, 228973883, 229099586, 2...","[-0.0027903197, 0.035919096, -0.03295671, -0.0..."
...,...,...,...,...,...
5416,seller requests buyer use seller,"[-1.3322289999999999, 0.16222735849056613, -0....","[-0.25, -0.1320754716981132, 0.042553191489361...","[198263962, 200651017, 209001692, 212786867, 1...","[-0.071547754, -0.007990354, 0.044830173, 0.01..."
5417,great home investors,"[0.2702147777777778, -0.23938986666666673, -0....","[-0.1, -0.19866666666666666, -0.08790322580645...","[197054770, 3232636, 151180267, 130595497, 532...","[-0.021865588, -0.05759102, -0.037234746, -0.0..."
5418,needs repairs,"[-0.7064920000000001, -0.2677977391304347, -0....","[-0.047619047619047616, -0.08695652173913043, ...","[230618952, 187242734, 196407010, 139519327, 5...","[-0.04354262, -0.037721384, 0.010220117, 0.044..."
5419,seller chooses attorney,"[0.06298180555555555, -0.2739625714285714, -0....","[0.02847222222222222, -0.2571428571428571, -0....","[77911179, 189586243, 180510904, 77958589, 854...","[-0.05809925, -0.012980465, -0.021596901, 0.03..."


In [115]:
noun_sentence_df_final_reindex_df.reset_index().to_feather('noun_sentence_df_final_reindex_df.fea')

In [116]:
import json
# create json object from dictionary
json = json.dumps(skill_group_full_final, indent = 4)
f = open("noun_phrases_full_final.json","w")
f.write(json)
f.close()

In [117]:
skill_group_full_final

{'new bathrooms': ['new bathrooms',
  'new kitchen bathrooms',
  'new vanities bathrooms',
  'new bathrooms'],
 'new vanities': ['new vanities', 'new vanities'],
 'quartz countertops': ['quartz countertops',
  'quartz countertops',
  'quartz countertops',
  'quartz countertop',
  'quartz counters',
  'kitchen quartz countertops',
  'quartz counter tops',
  'beautiful quartz countertops',
  'white quartz countertops',
  'new quartz countertops',
  'quartz countertops'],
 'top bottom': ['top bottom', 'top bottom'],
 'new kitchen cabinets': ['new kitchen cabinets',
  'new cabinets kitchen',
  'custom kitchen cabinets',
  'new kitchen cabinets',
  'new cabinets',
  'kitchen cabinets'],
 'new kitchen': ['new kitchen',
  'new custom kitchen',
  'new kitchen w',
  'new tile kitchen',
  'new kitchen',
  'new kitchen appliances'],
 'new windows': ['new windows',
  'new windows doors',
  'new thermopane windows',
  'new windows'],
 'new driveway': ['new driveway', 'new driveway'],
 'subway tile 